In [1]:
pip install vmdpy

Note: you may need to restart the kernel to use updated packages.


---
true implement
---
---

In [7]:
from pathlib import Path
import numpy as np
import pandas as pd

import mne
from mne.time_frequency import psd_array_welch

from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, StratifiedKFold, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import warnings
warnings.filterwarnings("ignore")

# ======================
# تنظیمات کلی
# ======================
DATA_FOLDER = "/kaggle/input/ahmadi-dataset"
INFO_CSV = f"{DATA_FOLDER}/subject-info.csv"

RESAMPLE_TO = 128        # می‌تونی 500 هم بذاری ولی کندتر می‌شه
RANDOM_STATE = 42

WINDOW_SEC = 2.0         # طول سگمنت (ثانیه)
OVERLAP_RATIO = 0.75     # ۷۵٪ overlap  → step = 0.5s

USE_BASELINE = True      # از EEG استراحت برای baseline correction استفاده کن


# ======================
# لود داده + baseline correction (اختیاری)
# ======================
def load_task_edf_with_baseline(folder_path, info_csv_path, resample_to=None,
                                use_baseline=True):
    """
    برای هر سوژه:
      - SubjectXX_1.edf (استراحت) و SubjectXX_2.edf (تسک) را می‌خوانیم
      - اگر use_baseline=True:
            task_data_bc = task_data - mean(rest_data, axis=time)
      - در نهایت، داده‌ی baseline-corrected حالت تسک را برمی‌گردانیم
    """
    folder = Path(folder_path)
    if not folder.is_dir():
        raise NotADirectoryError(f"{folder_path} is not a valid directory")

    info_df = pd.read_csv(info_csv_path)
    label_map = dict(zip(info_df["Subject"], info_df["Count quality"]))

    X_list = []
    y_list = []
    subjects = []
    sfreq = None

    for subj_row in info_df["Subject"]:
        subj_name = subj_row  # مثل Subject00, Subject01, ...
        task_file = folder / f"{subj_name}_2.edf"
        rest_file = folder / f"{subj_name}_1.edf"

        if not task_file.is_file():
            print(f"Task file not found for {subj_name}, skipping.")
            continue

        if subj_name not in label_map:
            print(f"Warning: {subj_name} not in subject-info, skipping.")
            continue

        print(f"Loading {subj_name}_1.edf (rest) and {subj_name}_2.edf (task)...")

        # --- read rest and task
        raw_task = mne.io.read_raw_edf(task_file, preload=True, verbose=False)

        if use_baseline:
            if rest_file.is_file():
                raw_rest = mne.io.read_raw_edf(rest_file, preload=True, verbose=False)
            else:
                print(f"Rest file not found for {subj_name}, baseline skipped for this subject.")
                raw_rest = None
        else:
            raw_rest = None

        # --- resample
        if resample_to is not None:
            raw_task.resample(resample_to)
            if raw_rest is not None:
                raw_rest.resample(resample_to)

        if sfreq is None:
            sfreq = raw_task.info["sfreq"]

        task_data = raw_task.get_data()  # (C, T_task)

        if use_baseline and raw_rest is not None:
            rest_data = raw_rest.get_data()  # (C, T_rest)
            # مطابق توضیح Oran & Yildirim: میانگین ولتاژ resting را کم می‌کنیم
            # Baseline Correction: average voltage in rest is subtracted from task. 
            baseline = rest_data.mean(axis=1, keepdims=True)
            task_data = task_data - baseline

        X_list.append(task_data)
        y_list.append(int(label_map[subj_name]))
        subjects.append(subj_name)

    if not X_list:
        raise ValueError("No task files loaded. Check folder path and file naming.")

    # همه سوژه‌ها را تا کوتاه‌ترین طول برش می‌دهیم
    lengths = [d.shape[1] for d in X_list]
    min_len = min(lengths)

    X = np.stack([d[:, :min_len] for d in X_list], axis=0)  # (N, C, T)
    y = np.array(y_list, dtype=int)
    subjects = np.array(subjects)

    print("Final tensor X shape (N, C, T):", X.shape)
    print("Labels y shape:", y.shape)

    return X, y, subjects, sfreq


# ======================
# Segment کردن با overlap درصدی
# ======================
def make_segments(X, y, subjects, sfreq, window_sec=2.0, overlap_ratio=0.75):
    """
    X: (N_subjects, C, T)
    خروجی:
      seg_X: (N_segments, C, T_seg)
      seg_y: (N_segments,)
      seg_subjects: (N_segments,)
    """
    N, C, T = X.shape
    win_size = int(window_sec * sfreq)
    step_sec = window_sec * (1.0 - overlap_ratio)
    step = max(1, int(step_sec * sfreq))

    seg_X_list = []
    seg_y_list = []
    seg_subj_list = []

    for i in range(N):
        data = X[i]          # (C, T)
        subj_label = y[i]
        subj_id = subjects[i]

        start = 0
        while start + win_size <= T:
            seg = data[:, start:start+win_size]  # (C, win_size)
            seg_X_list.append(seg)
            seg_y_list.append(subj_label)
            seg_subj_list.append(subj_id)
            start += step

    seg_X = np.stack(seg_X_list, axis=0)
    seg_y = np.array(seg_y_list, dtype=int)
    seg_subjects = np.array(seg_subj_list)

    print("Segmented data shape (N_segments, C, T_seg):", seg_X.shape)
    return seg_X, seg_y, seg_subjects


# ======================
# log-bandpower (absolute + relative) مثل قبل
# ======================
def compute_bandpower_features(seg_X, sfreq):
    """
    seg_X: (N_segments, C, T_seg)
    خروجی:
      feats: (N_segments, C * n_bands * 2)
    """
    band_defs = {
        "delta": (0.5, 4),
        "theta": (4, 8),
        "alpha": (8, 13),
        "beta":  (13, 30),
    }

    n_segments, C, T_seg = seg_X.shape
    feats_list = []

    for i in range(n_segments):
        data = seg_X[i]  # (C, T_seg)

        psd, freqs = psd_array_welch(
            data[np.newaxis, :, :],
            sfreq=sfreq,
            fmin=0.5,
            fmax=40,
            n_fft=T_seg,
            verbose=False
        )  # (1, C, n_freqs)

        psd = psd[0]  # (C, n_freqs)
        total_power = psd.sum(axis=1, keepdims=True) + 1e-12

        feat_abs = []
        feat_rel = []

        for (fmin, fmax) in band_defs.values():
            band_mask = (freqs >= fmin) & (freqs < fmax)
            band_power = psd[:, band_mask].mean(axis=1)        # absolute
            band_power_rel = band_power[:, None] / total_power  # relative

            feat_abs.append(band_power)
            feat_rel.append(band_power_rel[:, 0])

        feat_abs = np.concatenate(feat_abs, axis=0)  # (C * n_bands,)
        feat_rel = np.concatenate(feat_rel, axis=0)

        feat_seg = np.concatenate(
            [np.log10(feat_abs + 1e-12),
             np.log10(feat_rel + 1e-12)],
            axis=0
        )
        feats_list.append(feat_seg)

    feats = np.stack(feats_list, axis=0)
    print("Feature matrix shape:", feats.shape)
    return feats


# ======================
# ارزیابی ۱: LOSO روی سوژه‌ها (واقعی)
# ======================
def evaluate_loso(feats_all, seg_y, seg_subjects):
    unique_subjects = np.unique(seg_subjects)
    print("Unique subjects:", unique_subjects)

    seg_metrics = []
    subj_metrics = []

    for test_subj in unique_subjects:
        print("\n" + "=" * 60)
        print(f"Test subject: {test_subj}")

        test_mask = (seg_subjects == test_subj)
        train_mask = ~test_mask

        X_train = feats_all[train_mask]
        y_train = seg_y[train_mask]
        X_test = feats_all[test_mask]
        y_test = seg_y[test_mask]

        print("Train segments:", X_train.shape[0], " | Test segments:", X_test.shape[0])
        print("Train label distribution:", np.bincount(y_train))

        pipe = Pipeline([
            ("scaler", StandardScaler()),
            ("clf", SVC(kernel="rbf",
                        class_weight="balanced",
                        probability=False,
                        random_state=RANDOM_STATE))
        ])

        param_grid = {
            "clf__C": [1, 10],
            "clf__gamma": [0.01, 0.1]
        }

        cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

        grid = GridSearchCV(
            pipe,
            param_grid=param_grid,
            scoring="f1",
            cv=cv,
            n_jobs=-1,
            verbose=0
        )
        grid.fit(X_train, y_train)
        print("Best params:", grid.best_params_, "| best f1 (cv):", grid.best_score_)

        best_model = grid.best_estimator_

        # segment-level
        y_pred_seg = best_model.predict(X_test)
        acc_s = accuracy_score(y_test, y_pred_seg)
        prec_s = precision_score(y_test, y_pred_seg, zero_division=0)
        rec_s = recall_score(y_test, y_pred_seg, zero_division=0)
        f1_s = f1_score(y_test, y_pred_seg, zero_division=0)
        print(f"Segment-level -> acc: {acc_s:.4f}, prec: {prec_s:.4f}, rec: {rec_s:.4f}, f1: {f1_s:.4f}")
        seg_metrics.append([acc_s, prec_s, rec_s, f1_s])

        # subject-level (majority vote)
        true_label = y_test[0]
        pred_counts = np.bincount(y_pred_seg)
        pred_label = np.argmax(pred_counts)

        acc_subj = 1.0 if pred_label == true_label else 0.0
        print(f"Subject-level -> true: {true_label}, pred: {pred_label}, acc: {acc_subj:.4f}")

        subj_metrics.append([acc_subj, acc_subj, acc_subj, acc_subj])

    seg_metrics = np.array(seg_metrics)
    subj_metrics = np.array(subj_metrics)

    print("\n" + "=" * 60)
    print("Average SEGMENT-level (LOSO):")
    print(f"Accuracy : {seg_metrics[:,0].mean():.4f}")
    print(f"Precision: {seg_metrics[:,1].mean():.4f}")
    print(f"Recall   : {seg_metrics[:,2].mean():.4f}")
    print(f"F1-score : {seg_metrics[:,3].mean():.4f}")

    print("\n" + "=" * 60)
    print("Average SUBJECT-level (LOSO, majority vote):")
    print(f"Accuracy : {subj_metrics[:,0].mean():.4f}")
    print(f"Precision: {subj_metrics[:,1].mean():.4f}")
    print(f"Recall   : {subj_metrics[:,2].mean():.4f}")
    print(f"F1-score : {subj_metrics[:,3].mean():.4f}")


# ======================
# ارزیابی ۲: 10-fold CV روی همه‌ی سگمنت‌ها (مثل مقاله‌ها – subject-dependent)
# ======================
def evaluate_segment_cv(feats_all, seg_y):
    pipe = Pipeline([
        ("scaler", StandardScaler()),
        ("clf", SVC(kernel="rbf",
                    class_weight="balanced",
                    probability=False,
                    random_state=RANDOM_STATE))
    ])

    param_grid = {
        "clf__C": [1, 10],
        "clf__gamma": [0.01, 0.1]
    }

    cv_inner = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
    grid = GridSearchCV(
        pipe,
        param_grid=param_grid,
        scoring="f1",
        cv=cv_inner,
        n_jobs=-1,
        verbose=0
    )
    cv_outer = StratifiedKFold(n_splits=10, shuffle=True, random_state=RANDOM_STATE)

    scores_acc = []
    scores_f1 = []

    print("\n" + "=" * 60)
    print("10-fold CV over all segments (subject-dependent, شبیه اکثر مقاله‌ها)")

    for fold_idx, (train_idx, test_idx) in enumerate(cv_outer.split(feats_all, seg_y), start=1):
        X_train, X_test = feats_all[train_idx], feats_all[test_idx]
        y_train, y_test = seg_y[train_idx], seg_y[test_idx]

        grid.fit(X_train, y_train)
        best_model = grid.best_estimator_
        y_pred = best_model.predict(X_test)

        acc = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, zero_division=0)

        print(f"Fold {fold_idx}: acc={acc:.4f}, f1={f1:.4f}")
        scores_acc.append(acc)
        scores_f1.append(f1)

    print("\nAverage (10-fold CV, segment-level):")
    print(f"Accuracy: {np.mean(scores_acc):.4f} ± {np.std(scores_acc):.4f}")
    print(f"F1-score: {np.mean(scores_f1):.4f} ± {np.std(scores_f1):.4f}")


# ======================
# main
# ======================
def main():
    # 1) لود داده + baseline correction
    X, y, subjects, sfreq = load_task_edf_with_baseline(
        DATA_FOLDER,
        INFO_CSV,
        resample_to=RESAMPLE_TO,
        use_baseline=USE_BASELINE
    )

    # 2) سگمنت‌ها
    seg_X, seg_y, seg_subjects = make_segments(
        X, y, subjects,
        sfreq=sfreq,
        window_sec=WINDOW_SEC,
        overlap_ratio=OVERLAP_RATIO
    )

    # 3) فیچرها
    feats_all = compute_bandpower_features(seg_X, sfreq)

    # 4) ارزیابی واقعی (LOSO)
    evaluate_loso(feats_all, seg_y, seg_subjects)

    # 5) ارزیابی شبیه مقاله‌ها (10-fold CV روی سگمنت‌ها)
    evaluate_segment_cv(feats_all, seg_y)


if __name__ == "__main__":
    main()


Loading Subject00_1.edf (rest) and Subject00_2.edf (task)...
Loading Subject01_1.edf (rest) and Subject01_2.edf (task)...
Loading Subject02_1.edf (rest) and Subject02_2.edf (task)...
Loading Subject03_1.edf (rest) and Subject03_2.edf (task)...
Loading Subject04_1.edf (rest) and Subject04_2.edf (task)...
Loading Subject05_1.edf (rest) and Subject05_2.edf (task)...
Loading Subject06_1.edf (rest) and Subject06_2.edf (task)...
Loading Subject07_1.edf (rest) and Subject07_2.edf (task)...
Loading Subject08_1.edf (rest) and Subject08_2.edf (task)...
Loading Subject09_1.edf (rest) and Subject09_2.edf (task)...
Loading Subject10_1.edf (rest) and Subject10_2.edf (task)...
Loading Subject11_1.edf (rest) and Subject11_2.edf (task)...
Loading Subject12_1.edf (rest) and Subject12_2.edf (task)...
Loading Subject13_1.edf (rest) and Subject13_2.edf (task)...
Loading Subject14_1.edf (rest) and Subject14_2.edf (task)...
Loading Subject15_1.edf (rest) and Subject15_2.edf (task)...
Loading Subject16_1.edf 

In [14]:
# eegmat_data.py
from pathlib import Path
import numpy as np
import pandas as pd
import mne

import warnings
warnings.filterwarnings("ignore")

DATA_FOLDER = "/kaggle/input/ahmadi-dataset"
INFO_CSV = f"{DATA_FOLDER}/subject-info.csv"

RESAMPLE_TO = 128
WINDOW_SEC = 2.0
OVERLAP_RATIO = 0.75  # 75%

def load_task_edf_with_baseline(folder_path=DATA_FOLDER,
                                info_csv_path=INFO_CSV,
                                resample_to=RESAMPLE_TO,
                                use_baseline=True):
    folder = Path(folder_path)
    info_df = pd.read_csv(info_csv_path)
    label_map = dict(zip(info_df["Subject"], info_df["Count quality"]))

    X_list, y_list, subjects = [], [], []
    sfreq = None

    for subj_name in info_df["Subject"]:
        task_file = folder / f"{subj_name}_2.edf"
        rest_file = folder / f"{subj_name}_1.edf"

        if not task_file.is_file():
            print(f"Task file not found for {subj_name}, skipping.")
            continue

        if subj_name not in label_map:
            print(f"{subj_name} not in subject-info, skipping.")
            continue

        print(f"Loading {subj_name}_1.edf (rest) and {subj_name}_2.edf (task)...")
        raw_task = mne.io.read_raw_edf(task_file, preload=True, verbose=False)

        raw_rest = None
        if use_baseline and rest_file.is_file():
            raw_rest = mne.io.read_raw_edf(rest_file, preload=True, verbose=False)

        if resample_to is not None:
            raw_task.resample(resample_to)
            if raw_rest is not None:
                raw_rest.resample(resample_to)

        if sfreq is None:
            sfreq = raw_task.info["sfreq"]

        task_data = raw_task.get_data()  # (C, T)
        if use_baseline and raw_rest is not None:
            rest_data = raw_rest.get_data()
            baseline = rest_data.mean(axis=1, keepdims=True)
            task_data = task_data - baseline

        X_list.append(task_data)
        y_list.append(int(label_map[subj_name]))
        subjects.append(subj_name)

    if not X_list:
        raise ValueError("No task files loaded.")

    lengths = [d.shape[1] for d in X_list]
    min_len = min(lengths)
    X = np.stack([d[:, :min_len] for d in X_list], axis=0)  # (N, C, T)
    y = np.array(y_list, dtype=int)
    subjects = np.array(subjects)

    print("Final tensor X shape (N, C, T):", X.shape)
    print("Labels y shape:", y.shape)

    return X, y, subjects, sfreq


def make_segments(X, y, subjects, sfreq, window_sec=WINDOW_SEC, overlap_ratio=OVERLAP_RATIO):
    N, C, T = X.shape
    win_size = int(window_sec * sfreq)
    step_sec = window_sec * (1.0 - overlap_ratio)
    step = max(1, int(step_sec * sfreq))

    seg_X_list, seg_y_list, seg_subj_list = [], [], []

    for i in range(N):
        data = X[i]
        subj_label = y[i]
        subj_id = subjects[i]

        start = 0
        while start + win_size <= T:
            seg = data[:, start:start+win_size]
            seg_X_list.append(seg)
            seg_y_list.append(subj_label)
            seg_subj_list.append(subj_id)
            start += step

    seg_X = np.stack(seg_X_list, axis=0)
    seg_y = np.array(seg_y_list, dtype=int)
    seg_subjects = np.array(seg_subj_list)

    print("Segmented data shape:", seg_X.shape)
    return seg_X, seg_y, seg_subjects


In [9]:
pip install pyriemann


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.7/127.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 101.1 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
mkl-umath 0.1.1 requires numpy<1.27.0,>=1.26.4, but you have numpy 2.3.5 which is incompatible.
mkl-random 1.2.4 requires numpy<1.27.0,>=1.26.4, but you have numpy 2.3.5 which is incompatible.
mkl-fft 1.3.8 requires numpy<1.27.0,>=1.26.4, but you have numpy 2.3.5 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, b

In [15]:
# riemann_loso.py
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression

from pyriemann.estimation import Covariances
from pyriemann.tangentspace import TangentSpace
from pyriemann.utils.mean import mean_riemann

# from eegmat_data import load_task_edf_with_baseline, make_segments


RANDOM_STATE = 42


def compute_covariances(seg_X):
    """
    seg_X: (N_segments, C, T)
    returns: covs (N_segments, C, C)
    """
    cov_est = Covariances(estimator='oas')
    covs = cov_est.fit_transform(seg_X)
    return covs


def align_per_subject(covs, seg_subjects):
    """
    Riemannian centering per subject:
    cov_aligned = M_s^{-1/2} * C * M_s^{-1/2}
    """
    from scipy.linalg import fractional_matrix_power

    covs_aligned = np.zeros_like(covs)
    unique_subj = np.unique(seg_subjects)

    for subj in unique_subj:
        mask = (seg_subjects == subj)
        cov_subj = covs[mask]  # (N_subj, C, C)
        M = mean_riemann(cov_subj)  # (C, C)
        Minv_half = fractional_matrix_power(M, -0.5)
        for i_idx, c in zip(np.where(mask)[0], cov_subj):
            covs_aligned[i_idx] = Minv_half @ c @ Minv_half.T

    return covs_aligned


def evaluate_riemann_loso(use_alignment=False, clf_type="svm"):
    X, y, subjects, sfreq = load_task_edf_with_baseline()
    seg_X, seg_y, seg_subjects = make_segments(X, y, subjects, sfreq)

    covs = compute_covariances(seg_X)
    if use_alignment:
        print("Applying per-subject Riemannian alignment...")
        covs = align_per_subject(covs, seg_subjects)

    ts = TangentSpace()
    feats_all = ts.fit_transform(covs)  # (N_segments, n_features)

    unique_subjects = np.unique(seg_subjects)
    seg_metrics = []
    subj_metrics = []

    for test_subj in unique_subjects:
        print("\n" + "=" * 50)
        print("Test subject:", test_subj)

        test_mask = (seg_subjects == test_subj)
        train_mask = ~test_mask

        X_train = feats_all[train_mask]
        y_train = seg_y[train_mask]
        X_test = feats_all[test_mask]
        y_test = seg_y[test_mask]

        print("Train segments:", X_train.shape[0], "| Test segments:", X_test.shape[0])

        if clf_type == "svm":
            clf = SVC(kernel="rbf", class_weight="balanced", probability=False, random_state=RANDOM_STATE)
        else:
            clf = LogisticRegression(max_iter=2000, class_weight="balanced", random_state=RANDOM_STATE)

        clf.fit(X_train, y_train)
        y_pred_seg = clf.predict(X_test)

        acc_s = accuracy_score(y_test, y_pred_seg)
        prec_s = precision_score(y_test, y_pred_seg, zero_division=0)
        rec_s = recall_score(y_test, y_pred_seg, zero_division=0)
        f1_s = f1_score(y_test, y_pred_seg, zero_division=0)
        print(f"Segment-level -> acc: {acc_s:.4f}, prec: {prec_s:.4f}, rec: {rec_s:.4f}, f1: {f1_s:.4f}")
        seg_metrics.append([acc_s, prec_s, rec_s, f1_s])

        # subject-level majority vote
        true_label = y_test[0]
        counts = np.bincount(y_pred_seg)
        pred_label = np.argmax(counts)
        acc_subj = 1.0 if pred_label == true_label else 0.0
        print(f"Subject-level -> true: {true_label}, pred: {pred_label}, acc: {acc_subj:.4f}")
        subj_metrics.append([acc_subj, acc_subj, acc_subj, acc_subj])

    seg_metrics = np.array(seg_metrics)
    subj_metrics = np.array(subj_metrics)

    print("\n" + "=" * 50)
    print("Average SEGMENT-level (LOSO):")
    print("Accuracy :", seg_metrics[:, 0].mean())
    print("Precision:", seg_metrics[:, 1].mean())
    print("Recall   :", seg_metrics[:, 2].mean())
    print("F1-score :", seg_metrics[:, 3].mean())

    print("\nAverage SUBJECT-level (LOSO, majority vote):")
    print("Accuracy :", subj_metrics[:, 0].mean())


if __name__ == "__main__":
    print("=== Riemannian (no alignment) ===")
    evaluate_riemann_loso(use_alignment=False, clf_type="svm")

    print("\n\n=== Riemannian + per-subject alignment ===")
    evaluate_riemann_loso(use_alignment=True, clf_type="svm")


=== Riemannian (no alignment) ===
Loading Subject00_1.edf (rest) and Subject00_2.edf (task)...
Loading Subject01_1.edf (rest) and Subject01_2.edf (task)...
Loading Subject02_1.edf (rest) and Subject02_2.edf (task)...
Loading Subject03_1.edf (rest) and Subject03_2.edf (task)...
Loading Subject04_1.edf (rest) and Subject04_2.edf (task)...
Loading Subject05_1.edf (rest) and Subject05_2.edf (task)...
Loading Subject06_1.edf (rest) and Subject06_2.edf (task)...
Loading Subject07_1.edf (rest) and Subject07_2.edf (task)...
Loading Subject08_1.edf (rest) and Subject08_2.edf (task)...
Loading Subject09_1.edf (rest) and Subject09_2.edf (task)...
Loading Subject10_1.edf (rest) and Subject10_2.edf (task)...
Loading Subject11_1.edf (rest) and Subject11_2.edf (task)...
Loading Subject12_1.edf (rest) and Subject12_2.edf (task)...
Loading Subject13_1.edf (rest) and Subject13_2.edf (task)...
Loading Subject14_1.edf (rest) and Subject14_2.edf (task)...
Loading Subject15_1.edf (rest) and Subject15_2.edf 

In [19]:
pip install torch torchvision


Note: you may need to restart the kernel to use updated packages.


In [21]:
# dann_eegmat.py
import numpy as np
# from eegmat_data import load_task_edf_with_baseline, make_segments

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader


torch.backends.cudnn.enabled = False
torch.backends.cudnn.benchmark = False

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", DEVICE)

RANDOM_STATE = 42
torch.manual_seed(RANDOM_STATE)
np.random.seed(RANDOM_STATE)


# ---------- Dataset ----------
class EEGSegmentDataset(Dataset):
    def __init__(self, X, y, subj_ids):
        # X: (N, C, T)
        self.X = torch.from_numpy(X).float()
        self.y = torch.from_numpy(y).long()
        # subj_ids: mapping string -> int باید قبلش انجام بشه
        self.subj_ids = torch.from_numpy(subj_ids).long()

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx], self.subj_ids[idx]


# ---------- Gradient Reversal Layer ----------
class GradReverse(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x, lambd):
        ctx.lambd = lambd
        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        return -ctx.lambd * grad_output, None


def grad_reverse(x, lambd=1.0):
    return GradReverse.apply(x, lambd)


# ---------- Model ----------
class DANN_EEG(nn.Module):
    def __init__(self, n_channels, input_len, n_domains, feat_dim=128):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv1d(n_channels, 32, kernel_size=7, stride=1, padding=3),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.MaxPool1d(2),

            nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.MaxPool1d(2),

            nn.Conv1d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.AdaptiveAvgPool1d(1),  # -> (B, 128, 1)
        )
        self.feat = nn.Linear(128, feat_dim)

        self.label_head = nn.Sequential(
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(feat_dim, 1)
        )

        self.domain_head = nn.Sequential(
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(feat_dim, n_domains)
        )

    def forward(self, x, lambd=0.0):
        # x: (B, C, T)
        h = self.conv(x)          # (B, 128, 1)
        h = h.squeeze(-1)         # (B, 128)
        h = self.feat(h)          # (B, feat_dim)

        # label prediction
        logits_label = self.label_head(h).squeeze(-1)  # (B,)

        # domain prediction with GRL
        h_rev = grad_reverse(h, lambd)
        logits_domain = self.domain_head(h_rev)        # (B, n_domains)

        return logits_label, logits_domain


# ---------- Helper: subject mapping ----------
def encode_subjects(subj_array, train_subjects):
    """
    subj_array: (N_segments,) with string IDs
    train_subjects: unique strings used in this fold
    returns: domain_ids (int) for each segment, domain_label_map dict
    """
    subj_to_idx = {s: i for i, s in enumerate(train_subjects)}
    domain_ids = np.array([subj_to_idx[s] for s in subj_array])
    return domain_ids, subj_to_idx


# ---------- Training Loop ----------
def train_dann_for_fold(X_train, y_train, subj_train, X_val, y_val, subj_val,
                        num_epochs=15, batch_size=64):
    # Map subject strings to domain ids (only train subjects)
    unique_train_subj = np.unique(subj_train)
    train_domain_ids, subj_to_idx = encode_subjects(subj_train, unique_train_subj)

    # For validation segments, if subj not in train (shouldn't happen here for LOSO),
    # we skip domain loss; ساده‌ترین حالت: domain id = 0 (ignored).
    val_domain_ids = np.array([subj_to_idx.get(s, 0) for s in subj_val])

    train_ds = EEGSegmentDataset(X_train, y_train, train_domain_ids)
    val_ds = EEGSegmentDataset(X_val, y_val, val_domain_ids)

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, drop_last=True)
    val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False)

    n_channels = X_train.shape[1]
    input_len = X_train.shape[2]
    n_domains = len(unique_train_subj)

    model = DANN_EEG(n_channels, input_len, n_domains).to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)
    bce = nn.BCEWithLogitsLoss()
    ce = nn.CrossEntropyLoss()

    total_steps = num_epochs * len(train_loader)

    def calc_lambda(p):
        # p in [0,1], schedule از مقاله DANN
        return 2.0 / (1.0 + np.exp(-10 * p)) - 1.0

    step = 0
    for epoch in range(num_epochs):
        model.train()
        for xb, yb, db in train_loader:
            xb = xb.to(DEVICE)
            yb = yb.float().to(DEVICE)
            db = db.to(DEVICE)

            p = step / total_steps
            lambd = calc_lambda(p)

            logits_label, logits_domain = model(xb, lambd=lambd)

            loss_cls = bce(logits_label, yb)
            loss_dom = ce(logits_domain, db)

            loss = loss_cls + 0.1 * loss_dom  # وزن domain

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            step += 1

        # ساده: فقط یک پاس validation برای چاپ
        model.eval()
        with torch.no_grad():
            all_logits = []
            all_y = []
            for xb, yb, db in val_loader:
                xb = xb.to(DEVICE)
                yb = yb.float().to(DEVICE)
                lg, _ = model(xb, lambd=0.0)
                all_logits.append(lg.cpu())
                all_y.append(yb.cpu())
            if len(all_logits) > 0:
                logits = torch.cat(all_logits)
                ys = torch.cat(all_y)
                preds = (torch.sigmoid(logits) >= 0.5).long()
                acc = (preds == ys.long()).float().mean().item()
                print(f"Epoch {epoch+1}/{num_epochs}, val acc={acc:.4f}")

    return model


def evaluate_dann_loso(num_epochs=15):
    X, y, subjects, sfreq = load_task_edf_with_baseline()
    seg_X, seg_y, seg_subjects = make_segments(X, y, subjects, sfreq)

    unique_subjects = np.unique(seg_subjects)
    subj_metrics = []

    for test_subj in unique_subjects:
        print("\n" + "=" * 60)
        print("Test subject:", test_subj)

        test_mask = (seg_subjects == test_subj)
        train_mask = ~test_mask

        X_train = seg_X[train_mask]
        y_train = seg_y[train_mask]
        subj_train = seg_subjects[train_mask]

        X_test = seg_X[test_mask]
        y_test = seg_y[test_mask]
        subj_test = seg_subjects[test_mask]

        # split train -> train/val (مثلاً 90/10)
        n_train = X_train.shape[0]
        idx = np.arange(n_train)
        np.random.shuffle(idx)
        split = int(0.9 * n_train)
        tr_idx, val_idx = idx[:split], idx[split:]

        X_tr, y_tr, subj_tr = X_train[tr_idx], y_train[tr_idx], subj_train[tr_idx]
        X_val, y_val, subj_val = X_train[val_idx], y_train[val_idx], subj_train[val_idx]

        model = train_dann_for_fold(X_tr, y_tr, subj_tr, X_val, y_val, subj_val,
                                    num_epochs=num_epochs, batch_size=64)

        # test
        model.eval()
        ds_test = EEGSegmentDataset(X_test, y_test,
                                    np.zeros_like(y_test))  # domain ids irrelevant
        dl_test = DataLoader(ds_test, batch_size=64, shuffle=False)

        all_preds = []
        all_true = []
        with torch.no_grad():
            for xb, yb, db in dl_test:
                xb = xb.to(DEVICE)
                yb = yb.to(DEVICE)
                logits, _ = model(xb, lambd=0.0)
                preds = (torch.sigmoid(logits) >= 0.5).long()
                all_preds.append(preds.cpu().numpy())
                all_true.append(yb.cpu().numpy())

        y_pred_seg = np.concatenate(all_preds)
        y_true_seg = np.concatenate(all_true)

        # majority vote subject-level
        true_label = y_true_seg[0]
        counts = np.bincount(y_pred_seg)
        pred_label = np.argmax(counts)
        acc_subj = 1.0 if pred_label == true_label else 0.0
        print(f"Subject-level -> true: {true_label}, pred: {pred_label}, acc: {acc_subj:.4f}")
        subj_metrics.append(acc_subj)

    subj_metrics = np.array(subj_metrics)
    print("\n" + "=" * 60)
    print("Average SUBJECT-level (LOSO, DANN):", subj_metrics.mean())


if __name__ == "__main__":
    evaluate_dann_loso(num_epochs=10)


Using device: cuda
Loading Subject00_1.edf (rest) and Subject00_2.edf (task)...
Loading Subject01_1.edf (rest) and Subject01_2.edf (task)...
Loading Subject02_1.edf (rest) and Subject02_2.edf (task)...
Loading Subject03_1.edf (rest) and Subject03_2.edf (task)...
Loading Subject04_1.edf (rest) and Subject04_2.edf (task)...
Loading Subject05_1.edf (rest) and Subject05_2.edf (task)...
Loading Subject06_1.edf (rest) and Subject06_2.edf (task)...
Loading Subject07_1.edf (rest) and Subject07_2.edf (task)...
Loading Subject08_1.edf (rest) and Subject08_2.edf (task)...
Loading Subject09_1.edf (rest) and Subject09_2.edf (task)...
Loading Subject10_1.edf (rest) and Subject10_2.edf (task)...
Loading Subject11_1.edf (rest) and Subject11_2.edf (task)...
Loading Subject12_1.edf (rest) and Subject12_2.edf (task)...
Loading Subject13_1.edf (rest) and Subject13_2.edf (task)...
Loading Subject14_1.edf (rest) and Subject14_2.edf (task)...
Loading Subject15_1.edf (rest) and Subject15_2.edf (task)...
Loadi

In [5]:
# ================================
#   SimCLR-style SSL + LOSO + SVM
#       روی EEGMAT - GPU if avail
# ================================

from pathlib import Path
import numpy as np
import pandas as pd
import mne
import warnings
warnings.filterwarnings("ignore")

# --------------------------------
# تنظیمات دیتاست
# --------------------------------
DATA_FOLDER = "/kaggle/input/ahmadi-dataset"   # اگر مسیرت فرق دارد، این را عوض کن
INFO_CSV = f"{DATA_FOLDER}/subject-info.csv"

RESAMPLE_TO = 128
WINDOW_SEC = 2.0
OVERLAP_RATIO = 0.75  # 75% overlap

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# --------------------------------
# ۱) لود دیتاست + baseline از rest
# --------------------------------
def load_task_edf_with_baseline(folder_path=DATA_FOLDER,
                                info_csv_path=INFO_CSV,
                                resample_to=RESAMPLE_TO,
                                use_baseline=True):
    folder = Path(folder_path)
    info_df = pd.read_csv(info_csv_path)
    label_map = dict(zip(info_df["Subject"], info_df["Count quality"]))

    X_list, y_list, subjects = [], [], []
    sfreq = None

    for subj_name in info_df["Subject"]:
        task_file = folder / f"{subj_name}_2.edf"
        rest_file = folder / f"{subj_name}_1.edf"

        if not task_file.is_file():
            print(f"Task file not found for {subj_name}, skipping.")
            continue

        if subj_name not in label_map:
            print(f"{subj_name} not in subject-info, skipping.")
            continue

        print(f"Loading {subj_name}_1.edf (rest) and {subj_name}_2.edf (task)...")
        raw_task = mne.io.read_raw_edf(task_file, preload=True, verbose=False)

        raw_rest = None
        if use_baseline and rest_file.is_file():
            raw_rest = mne.io.read_raw_edf(rest_file, preload=True, verbose=False)

        # resample
        if resample_to is not None:
            raw_task.resample(resample_to)
            if raw_rest is not None:
                raw_rest.resample(resample_to)

        if sfreq is None:
            sfreq = raw_task.info["sfreq"]

        task_data = raw_task.get_data()  # (C, T)

        if use_baseline and raw_rest is not None:
            rest_data = raw_rest.get_data()
            baseline = rest_data.mean(axis=1, keepdims=True)
            task_data = task_data - baseline

        X_list.append(task_data)
        y_list.append(int(label_map[subj_name]))
        subjects.append(subj_name)

    if not X_list:
        raise ValueError("No task files loaded.")

    lengths = [d.shape[1] for d in X_list]
    min_len = min(lengths)
    X = np.stack([d[:, :min_len] for d in X_list], axis=0)  # (N, C, T)
    y = np.array(y_list, dtype=int)
    subjects = np.array(subjects)

    print("Final tensor X shape (N, C, T):", X.shape)
    print("Labels y shape:", y.shape)

    return X, y, subjects, sfreq


# --------------------------------
# ۲) سگمنت‌کردن سیگنال‌ها
# --------------------------------
def make_segments(X, y, subjects, sfreq,
                  window_sec=WINDOW_SEC,
                  overlap_ratio=OVERLAP_RATIO):
    N, C, T = X.shape
    win_size = int(window_sec * sfreq)
    step_sec = window_sec * (1.0 - overlap_ratio)
    step = max(1, int(step_sec * sfreq))

    seg_X_list, seg_y_list, seg_subj_list = [], [], []

    for i in range(N):
        data = X[i]
        subj_label = y[i]
        subj_id = subjects[i]

        start = 0
        while start + win_size <= T:
            seg = data[:, start:start+win_size]
            seg_X_list.append(seg)
            seg_y_list.append(subj_label)
            seg_subj_list.append(subj_id)
            start += step

    seg_X = np.stack(seg_X_list, axis=0)
    seg_y = np.array(seg_y_list, dtype=int)
    seg_subjects = np.array(seg_subj_list)

    print("Segmented data shape:", seg_X.shape)
    return seg_X, seg_y, seg_subjects


# --------------------------------
# ۳) بخش PyTorch (SSL + Encoder)
# --------------------------------
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

# cuDNN را فعال می‌گذاریم (برای سرعت)، فقط TF32 را خاموش می‌کنیم برای پایداری
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = False
torch.backends.cudnn.allow_tf32 = False

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", DEVICE)

torch.manual_seed(RANDOM_STATE)


# --------- Augmentations برای EEG (ملایم) ---------
class EEGAugment:
    def __init__(self, jitter_std=0.005, time_shift=0.05, dropout_p=0.05):
        self.jitter_std = jitter_std
        self.time_shift = time_shift
        self.dropout_p = dropout_p

    def __call__(self, x):
        # x: (C, T) tensor
        x = self.jitter(x)
        x = self.time_shift_aug(x)
        x = self.channel_dropout(x)
        return x

    def jitter(self, x):
        noise = torch.randn_like(x) * self.jitter_std
        return x + noise

    def time_shift_aug(self, x):
        T = x.shape[1]
        if T <= 1:
            return x
        shift = int(self.time_shift * T)
        if shift == 0:
            return x
        k = np.random.randint(-shift, shift + 1)
        return torch.roll(x, shifts=k, dims=1)

    def channel_dropout(self, x):
        # dropout روی کانال‌ها
        mask = (torch.rand(x.shape[0], 1, device=x.device) > self.dropout_p).float()
        return x * mask


# --------- Dataset برای SSL ---------
class EEGSSL_Dataset(Dataset):
    def __init__(self, X, augment):
        # X: (N, C, T)
        self.X = torch.from_numpy(X).float()
        self.augment = augment

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        x = self.X[idx]  # (C, T)
        v1 = self.augment(x)
        v2 = self.augment(x)
        return v1, v2


# --------- Encoder CNN ---------
class EEGEncoder(nn.Module):
    def __init__(self, n_channels, input_len, feat_dim=128):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv1d(n_channels, 32, kernel_size=7, padding=3),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.MaxPool1d(2),

            nn.Conv1d(32, 64, kernel_size=5, padding=2),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.MaxPool1d(2),

            nn.Conv1d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.AdaptiveAvgPool1d(1),
        )
        self.fc = nn.Linear(128, feat_dim)

    def forward(self, x):
        # x: (B, C, T)
        h = self.conv(x).squeeze(-1)  # (B, 128)
        z = self.fc(h)                # (B, feat_dim)
        return F.normalize(z, dim=1)  # نرمال‌سازی روی بردار embedding


# --------- NT-Xent (SimCLR استاندارد) ---------
def nt_xent_loss(z1, z2, temperature=0.1):
    """
    z1, z2: (B, D), normalized embeddings
    SimCLR NT-Xent: هر نمونه در view1 پارتنرش در view2 است و برعکس.
    """
    B = z1.size(0)

    # ۲B تا embedding پشت سر هم
    z = torch.cat([z1, z2], dim=0)         # (2B, D)

    # cosine similarity بین همه‌ی جفت‌ها: (2B, 2B)
    sim = F.cosine_similarity(
        z.unsqueeze(1),  # (2B, 1, D)
        z.unsqueeze(0),  # (1, 2B, D)
        dim=2
    )  # -> (2B, 2B)

    # positive index:
    # view1[i] ↔ view2[i]  → برای i در [0..B-1]
    labels = torch.arange(B, device=z.device)
    labels = torch.cat([labels + B, labels], dim=0)   # (2B,)

    # self-sim رو حذف می‌کنیم (قطر)
    mask = torch.eye(2 * B, dtype=torch.bool, device=z.device)
    sim = sim.masked_fill(mask, -1e9)

    # scale by temperature و cross-entropy
    sim = sim / temperature
    loss = F.cross_entropy(sim, labels)
    return loss


# --------- Pretraining SimCLR ---------
def pretrain_simclr(seg_X, num_epochs=20, batch_size=128, feat_dim=128):
    augment = EEGAugment()
    ds_ssl = EEGSSL_Dataset(seg_X, augment)
    dl_ssl = DataLoader(ds_ssl, batch_size=batch_size,
                        shuffle=True, drop_last=True)

    n_channels = seg_X.shape[1]
    input_len = seg_X.shape[2]
    encoder = EEGEncoder(n_channels, input_len,
                         feat_dim=feat_dim).to(DEVICE)

    optimizer = torch.optim.Adam(encoder.parameters(),
                                 lr=1e-3, weight_decay=1e-4)

    for epoch in range(num_epochs):
        encoder.train()
        total_loss = 0.0
        for v1, v2 in dl_ssl:
            v1 = v1.to(DEVICE)
            v2 = v2.to(DEVICE)

            z1 = encoder(v1)
            z2 = encoder(v2)

            loss = nt_xent_loss(z1, z2, temperature=0.1)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(dl_ssl)
        print(f"[SSL] Epoch {epoch+1}/{num_epochs}, loss={avg_loss:.4f}")

    return encoder


# --------- استخراج embedding ---------
def extract_embeddings(encoder, seg_X, batch_size=256):
    encoder.eval()
    ds = torch.from_numpy(seg_X).float()
    all_emb = []
    with torch.no_grad():
        for i in range(0, ds.shape[0], batch_size):
            xb = ds[i:i+batch_size].to(DEVICE)
            z = encoder(xb)
            all_emb.append(z.cpu().numpy())
    return np.concatenate(all_emb, axis=0)


# --------- LOSO Evaluation با SVM + StandardScaler ---------
def evaluate_ssl_loso(encoder, seg_X, seg_y, seg_subjects):
    feats_all = extract_embeddings(encoder, seg_X)
    unique_subjects = np.unique(seg_subjects)
    subj_accs = []

    for test_subj in unique_subjects:
        print("\n" + "=" * 50)
        print("Test subject:", test_subj)

        test_mask = (seg_subjects == test_subj)
        train_mask = ~test_mask

        X_train = feats_all[train_mask]
        y_train = seg_y[train_mask]
        X_test = feats_all[test_mask]
        y_test = seg_y[test_mask]

        print("Train segments:", X_train.shape[0],
              "| Test segments:", X_test.shape[0])

        clf = make_pipeline(
            StandardScaler(),
            SVC(kernel="rbf", class_weight="balanced",
                random_state=RANDOM_STATE)
        )
        clf.fit(X_train, y_train)
        y_pred_seg = clf.predict(X_test)

        # subject-level majority vote
        true_label = y_test[0]
        counts = np.bincount(y_pred_seg)
        pred_label = np.argmax(counts)
        acc_subj = 1.0 if pred_label == true_label else 0.0
        print(f"Subject-level -> true: {true_label}, "
              f"pred: {pred_label}, acc: {acc_subj:.4f}")
        subj_accs.append(acc_subj)

    subj_accs = np.array(subj_accs)
    print("\n" + "=" * 50)
    print("Average SUBJECT-level (LOSO, SSL+SVM):",
          subj_accs.mean())


# --------------------------------
# main
# --------------------------------
def main():
    X, y, subjects, sfreq = load_task_edf_with_baseline()
    seg_X, seg_y, seg_subjects = make_segments(X, y, subjects, sfreq)

    print("Pretraining SimCLR-style SSL on all segments...")
    encoder = pretrain_simclr(seg_X,
                              num_epochs=20,   # برای تست سریع می‌تونی 10 بذاری
                              batch_size=128,
                              feat_dim=128)

    print("Evaluating LOSO using frozen encoder + SVM...")
    evaluate_ssl_loso(encoder, seg_X, seg_y, seg_subjects)


if __name__ == "__main__":
    main()


Using device: cuda
Loading Subject00_1.edf (rest) and Subject00_2.edf (task)...
Loading Subject01_1.edf (rest) and Subject01_2.edf (task)...
Loading Subject02_1.edf (rest) and Subject02_2.edf (task)...
Loading Subject03_1.edf (rest) and Subject03_2.edf (task)...
Loading Subject04_1.edf (rest) and Subject04_2.edf (task)...
Loading Subject05_1.edf (rest) and Subject05_2.edf (task)...
Loading Subject06_1.edf (rest) and Subject06_2.edf (task)...
Loading Subject07_1.edf (rest) and Subject07_2.edf (task)...
Loading Subject08_1.edf (rest) and Subject08_2.edf (task)...
Loading Subject09_1.edf (rest) and Subject09_2.edf (task)...
Loading Subject10_1.edf (rest) and Subject10_2.edf (task)...
Loading Subject11_1.edf (rest) and Subject11_2.edf (task)...
Loading Subject12_1.edf (rest) and Subject12_2.edf (task)...
Loading Subject13_1.edf (rest) and Subject13_2.edf (task)...
Loading Subject14_1.edf (rest) and Subject14_2.edf (task)...
Loading Subject15_1.edf (rest) and Subject15_2.edf (task)...
Loadi

---
thst
---
---

In [ ]:
# ============================================
#   Dual-Path CNN (Temporal + Channel Branch)
#   LOSO روی EEGMAT با class-weight و val سوژه‌ای
# ============================================

from pathlib import Path
import numpy as np
import pandas as pd
import mne
import warnings
warnings.filterwarnings("ignore")

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# -----------------------------
# تنظیمات دیتاست و مدل
# -----------------------------
DATA_FOLDER = "/kaggle/input/ahmadi-dataset"    # مسیر دیتاست EEGMAT
INFO_CSV = f"{DATA_FOLDER}/subject-info.csv"

RESAMPLE_TO = 128         # Hz
WINDOW_SEC = 2.0          # طول سگمنت (ثانیه)
OVERLAP_RATIO = 0.75      # درصد overlap بین سگمنت‌ها

BATCH_SIZE = 64
NUM_EPOCHS = 10           # می‌تونی بعداً بیشترش کنی (مثلاً 20)
LR = 1e-3
WEIGHT_DECAY = 1e-4
RANDOM_STATE = 42

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", DEVICE)
if DEVICE.type == "cuda":
    print("CUDA version:", torch.version.cuda)
    print("GPU name:", torch.cuda.get_device_name(0))

torch.manual_seed(RANDOM_STATE)
np.random.seed(RANDOM_STATE)

# -----------------------------
# ۱) لود EEG + baseline از rest
# -----------------------------
def load_task_edf_with_baseline(folder_path=DATA_FOLDER,
                                info_csv_path=INFO_CSV,
                                resample_to=RESAMPLE_TO,
                                use_baseline=True):
    folder = Path(folder_path)
    info_df = pd.read_csv(info_csv_path)
    label_map = dict(zip(info_df["Subject"], info_df["Count quality"]))

    X_list, y_list, subjects = [], [], []
    sfreq = None

    for subj_name in info_df["Subject"]:
        task_file = folder / f"{subj_name}_2.edf"
        rest_file = folder / f"{subj_name}_1.edf"

        if not task_file.is_file():
            print(f"Task file not found for {subj_name}, skipping.")
            continue

        if subj_name not in label_map:
            print(f"{subj_name} not in subject-info, skipping.")
            continue

        print(f"Loading {subj_name}_1.edf (rest) and {subj_name}_2.edf (task)...")
        raw_task = mne.io.read_raw_edf(task_file, preload=True, verbose=False)

        raw_rest = None
        if use_baseline and rest_file.is_file():
            raw_rest = mne.io.read_raw_edf(rest_file, preload=True, verbose=False)

        if resample_to is not None:
            raw_task.resample(resample_to)
            if raw_rest is not None:
                raw_rest.resample(resample_to)

        if sfreq is None:
            sfreq = raw_task.info["sfreq"]

        task_data = raw_task.get_data()  # (C, T)

        if use_baseline and raw_rest is not None:
            rest_data = raw_rest.get_data()
            baseline = rest_data.mean(axis=1, keepdims=True)  # (C,1)
            task_data = task_data - baseline

        X_list.append(task_data)
        y_list.append(int(label_map[subj_name]))
        subjects.append(subj_name)

    if not X_list:
        raise ValueError("No task files loaded.")

    lengths = [d.shape[1] for d in X_list]
    min_len = min(lengths)
    X = np.stack([d[:, :min_len] for d in X_list], axis=0)  # (N, C, T)
    y = np.array(y_list, dtype=int)
    subjects = np.array(subjects)

    print("Final tensor X shape (N, C, T):", X.shape)
    print("Labels y shape:", y.shape)

    return X, y, subjects, sfreq


# -----------------------------
# ۲) سگمنت کردن سیگنال‌ها
# -----------------------------
def make_segments(X, y, subjects, sfreq,
                  window_sec=WINDOW_SEC,
                  overlap_ratio=OVERLAP_RATIO):
    N, C, T = X.shape
    win_size = int(window_sec * sfreq)
    step_sec = window_sec * (1.0 - overlap_ratio)
    step = max(1, int(step_sec * sfreq))

    seg_X_list, seg_y_list, seg_subj_list = [], [], []

    for i in range(N):
        data = X[i]
        subj_label = y[i]
        subj_id = subjects[i]

        start = 0
        while start + win_size <= T:
            seg = data[:, start:start+win_size]     # (C, win_size)
            seg_X_list.append(seg)
            seg_y_list.append(subj_label)
            seg_subj_list.append(subj_id)
            start += step

    seg_X = np.stack(seg_X_list, axis=0)       # (N_seg, C, win_size)
    seg_y = np.array(seg_y_list, dtype=int)
    seg_subj = np.array(seg_subj_list)

    print("Segmented data shape:", seg_X.shape)
    return seg_X, seg_y, seg_subj


# -----------------------------
# ۳) نرمال‌سازی بر اساس train
# -----------------------------
def standardize_train_val_test(X_train, X_val, X_test):
    mean = X_train.mean()
    std = X_train.std()
    if std == 0:
        std = 1.0
    X_train_norm = (X_train - mean) / std
    X_val_norm = (X_val - mean) / std
    X_test_norm = (X_test - mean) / std
    return (X_train_norm.astype("float32"),
            X_val_norm.astype("float32"),
            X_test_norm.astype("float32"))


# -----------------------------
# ۴) Dataset برای PyTorch
# -----------------------------
class EEGSegDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X).float()
        self.y = torch.from_numpy(y).long()

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


# -----------------------------
# ۵) Dual-Path CNN مدل
# -----------------------------
class DualPathEEGNet(nn.Module):
    """
    شاخه اول: روی (C, T) → فیچرهای زمانی (Temporal)
    شاخه دوم: روی (T, C) → Conv روی Channel → فیچرهای فضایی/چنلی
    """
    def __init__(self, n_channels, n_timepoints,
                 temporal_feat_dim=128, spatial_feat_dim=128,
                 hidden_dim=128, n_classes=2):
        super().__init__()

        # --- Temporal Branch (B, C, T) ---
        self.temporal_branch = nn.Sequential(
            nn.Conv1d(n_channels, 64, kernel_size=7, padding=3),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.MaxPool1d(2),

            nn.Conv1d(64, 128, kernel_size=5, padding=2),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.MaxPool1d(2),

            nn.Conv1d(128, temporal_feat_dim, kernel_size=3, padding=1),
            nn.BatchNorm1d(temporal_feat_dim),
            nn.ReLU(),
            nn.AdaptiveAvgPool1d(1)  # → (B, temporal_feat_dim, 1)
        )

        # --- Spatial/Channel Branch ---
        # x: (B, C, T) → (B, T, C) → (B*T, 1, C)
        self.spatial_conv = nn.Sequential(
            nn.Conv1d(1, 32, kernel_size=3, padding=1),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.MaxPool1d(2),

            nn.Conv1d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.MaxPool1d(2),

            nn.Conv1d(64, spatial_feat_dim, kernel_size=3, padding=1),
            nn.BatchNorm1d(spatial_feat_dim),
            nn.ReLU(),
            nn.AdaptiveAvgPool1d(1)  # → (B*T, spatial_feat_dim, 1)
        )

        fused_dim = temporal_feat_dim + spatial_feat_dim
        self.fusion_dropout = nn.Dropout(0.5)
        self.classifier = nn.Sequential(
            nn.Linear(fused_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(hidden_dim, n_classes)
        )

    def forward(self, x):
        """
        x: (B, C, T)
        """
        B, C, T = x.shape

        # --- Temporal branch ---
        t_feat = self.temporal_branch(x)          # (B, temporal_feat_dim, 1)
        t_feat = t_feat.squeeze(-1)               # (B, temporal_feat_dim)

        # --- Spatial branch ---
        x_tc = x.transpose(1, 2)                  # (B, T, C)
        x_tc_flat = x_tc.reshape(B * T, 1, C)     # (B*T, 1, C)
        s_feat = self.spatial_conv(x_tc_flat)     # (B*T, spatial_feat_dim, 1)
        s_feat = s_feat.squeeze(-1)               # (B*T, spatial_feat_dim)
        s_feat = s_feat.reshape(B, T, -1)         # (B, T, spatial_feat_dim)
        s_feat = s_feat.mean(dim=1)               # (B, spatial_feat_dim)

        fused = torch.cat([t_feat, s_feat], dim=1)  # (B, fused_dim)
        fused = self.fusion_dropout(fused)
        logits = self.classifier(fused)           # (B, n_classes)
        return logits


# -----------------------------
# ۶) train و evaluate روی یک fold
# -----------------------------
def train_one_fold(model, train_loader, val_loader,
                   num_epochs=NUM_EPOCHS,
                   lr=LR, weight_decay=WEIGHT_DECAY,
                   device=DEVICE,
                   class_weights=None):
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr,
                                 weight_decay=weight_decay)

    if class_weights is not None:
        criterion = nn.CrossEntropyLoss(weight=class_weights)
    else:
        criterion = nn.CrossEntropyLoss()

    best_val_f1 = -1.0
    best_state = None

    for epoch in range(num_epochs):
        # ----- train -----
        model.train()
        total_loss = 0.0
        for xb, yb in train_loader:
            xb = xb.to(device)
            yb = yb.to(device)

            optimizer.zero_grad()
            logits = model(xb)
            loss = criterion(logits, yb)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)

        # ----- validation -----
        model.eval()
        all_y = []
        all_pred = []
        with torch.no_grad():
            for xb, yb in val_loader:
                xb = xb.to(device)
                yb = yb.to(device)

                logits = model(xb)
                preds = torch.argmax(logits, dim=1)

                all_y.append(yb.cpu().numpy())
                all_pred.append(preds.cpu().numpy())

        all_y = np.concatenate(all_y)
        all_pred = np.concatenate(all_pred)

        acc = accuracy_score(all_y, all_pred)
        prec = precision_score(all_y, all_pred, zero_division=0)
        rec = recall_score(all_y, all_pred, zero_division=0)
        f1 = f1_score(all_y, all_pred, zero_division=0)

        print(f"Epoch {epoch+1}/{num_epochs} | "
              f"train_loss={avg_loss:.4f} | "
              f"val_acc={acc:.4f}, val_f1={f1:.4f}")

        if f1 > best_val_f1:
            best_val_f1 = f1
            best_state = model.state_dict()

    if best_state is not None:
        model.load_state_dict(best_state)

    return model


def evaluate_on_segments(model, loader, device=DEVICE):
    model.eval()
    all_y, all_pred = [], []
    with torch.no_grad():
        for xb, yb in loader:
            xb = xb.to(device)
            yb = yb.to(device)
            logits = model(xb)
            preds = torch.argmax(logits, dim=1)
            all_y.append(yb.cpu().numpy())
            all_pred.append(preds.cpu().numpy())

    all_y = np.concatenate(all_y)
    all_pred = np.concatenate(all_pred)

    acc = accuracy_score(all_y, all_pred)
    prec = precision_score(all_y, all_pred, zero_division=0)
    rec = recall_score(all_y, all_pred, zero_division=0)
    f1 = f1_score(all_y, all_pred, zero_division=0)
    return acc, prec, rec, f1, all_y, all_pred


# -----------------------------
# ۷) LOSO evaluation با val سوژه‌ای
# -----------------------------
def loso_dualpath(X_seg, y_seg, subj_seg):
    unique_subjects = np.unique(subj_seg)
    results_seg = []
    results_subj = []

    N_seg, C, T = X_seg.shape
    print("Segmented data shape:", X_seg.shape)

    for fold_idx, test_subj in enumerate(unique_subjects):
        print("\n" + "=" * 60)
        print("Test subject:", test_subj)

        is_test = (subj_seg == test_subj)
        is_train_all = ~is_test

        # سوژه‌های train
        train_subjects = np.unique(subj_seg[is_train_all])

        # سوژه‌های val (20% از train subjects)
        rng = np.random.RandomState(RANDOM_STATE + fold_idx)
        perm_subj = rng.permutation(train_subjects)
        n_val_subj = max(1, int(0.2 * len(train_subjects)))
        val_subj = perm_subj[:n_val_subj]
        real_train_subj = perm_subj[n_val_subj:]

        is_val = np.isin(subj_seg, val_subj) & is_train_all
        is_train = np.isin(subj_seg, real_train_subj)

        X_train = X_seg[is_train]
        y_train = y_seg[is_train]
        X_val = X_seg[is_val]
        y_val = y_seg[is_val]
        X_test = X_seg[is_test]
        y_test = y_seg[is_test]

        print(f"Train segments: {X_train.shape[0]} | "
              f"Val segments: {X_val.shape[0]} | "
              f"Test segments: {X_test.shape[0]}")

        # نرمال‌سازی
        X_train_norm, X_val_norm, X_test_norm = standardize_train_val_test(
            X_train, X_val, X_test
        )

        # class weights از روی y_train
        class_counts = np.bincount(y_train, minlength=2)
        class_counts[class_counts == 0] = 1
        total = class_counts.sum()
        num_classes = len(class_counts)
        weights_np = total / (num_classes * class_counts.astype(np.float32))
        class_weights = torch.tensor(weights_np, dtype=torch.float32).to(DEVICE)
        print("Class counts (train):", class_counts, "-> weights:", weights_np)

        train_ds = EEGSegDataset(X_train_norm, y_train)
        val_ds = EEGSegDataset(X_val_norm, y_val)
        test_ds = EEGSegDataset(X_test_norm, y_test)

        train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE,
                                  shuffle=True, drop_last=False)
        val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE,
                                shuffle=False, drop_last=False)
        test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE,
                                 shuffle=False, drop_last=False)

        model = DualPathEEGNet(
            n_channels=C,
            n_timepoints=T,
            temporal_feat_dim=128,
            spatial_feat_dim=128,
            hidden_dim=128,
            n_classes=2
        )

        model = train_one_fold(
            model,
            train_loader,
            val_loader,
            num_epochs=NUM_EPOCHS,
            lr=LR,
            weight_decay=WEIGHT_DECAY,
            device=DEVICE,
            class_weights=class_weights
        )

        acc_s, prec_s, rec_s, f1_s, y_true_seg, y_pred_seg = evaluate_on_segments(
            model, test_loader, device=DEVICE
        )

        print(f"Segment-level metrics -> acc: {acc_s:.4f}, "
              f"prec: {prec_s:.4f}, rec: {rec_s:.4f}, f1: {f1_s:.4f}")
        results_seg.append([acc_s, prec_s, rec_s, f1_s])

        # subject-level majority vote
        true_label = int(y_test[0])
        counts = np.bincount(y_pred_seg)
        pred_label = int(np.argmax(counts))
        acc_subj = 1.0 if pred_label == true_label else 0.0
        print(f"Subject-level prediction -> true: {true_label}, "
              f"pred: {pred_label}, acc: {acc_subj:.4f}")
        results_subj.append(acc_subj)

    results_seg = np.array(results_seg)
    subj_accs = np.array(results_subj)

    print("\n" + "=" * 60)
    print("Average SEGMENT-level (LOSO):")
    print(f"Accuracy : {results_seg[:,0].mean():.4f}")
    print(f"Precision: {results_seg[:,1].mean():.4f}")
    print(f"Recall   : {results_seg[:,2].mean():.4f}")
    print(f"F1-score : {results_seg[:,3].mean():.4f}")

    print("\nAverage SUBJECT-level (LOSO, majority vote):")
    print(f"Accuracy : {subj_accs.mean():.4f}")


# -----------------------------
# ۸) main
# -----------------------------
def main():
    X, y, subjects, sfreq = load_task_edf_with_baseline()
    X_seg, y_seg, subj_seg = make_segments(X, y, subjects, sfreq)
    loso_dualpath(X_seg, y_seg, subj_seg)


if __name__ == "__main__":
    main()


Using device: cuda
CUDA version: 12.4
GPU name: Tesla P100-PCIE-16GB
Loading Subject00_1.edf (rest) and Subject00_2.edf (task)...
Loading Subject01_1.edf (rest) and Subject01_2.edf (task)...
Loading Subject02_1.edf (rest) and Subject02_2.edf (task)...
Loading Subject03_1.edf (rest) and Subject03_2.edf (task)...
Loading Subject04_1.edf (rest) and Subject04_2.edf (task)...
Loading Subject05_1.edf (rest) and Subject05_2.edf (task)...
Loading Subject06_1.edf (rest) and Subject06_2.edf (task)...
Loading Subject07_1.edf (rest) and Subject07_2.edf (task)...
Loading Subject08_1.edf (rest) and Subject08_2.edf (task)...
Loading Subject09_1.edf (rest) and Subject09_2.edf (task)...
Loading Subject10_1.edf (rest) and Subject10_2.edf (task)...
Loading Subject11_1.edf (rest) and Subject11_2.edf (task)...
Loading Subject12_1.edf (rest) and Subject12_2.edf (task)...
Loading Subject13_1.edf (rest) and Subject13_2.edf (task)...
Loading Subject14_1.edf (rest) and Subject14_2.edf (task)...
Loading Subject1

---
bi lstm
---
---

In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import random
import warnings
warnings.filterwarnings("ignore")

import mne
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# ======================
# تنظیمات کلی
# ======================
DATA_FOLDER = "/kaggle/input/ahmadi-dataset"
INFO_CSV = f"{DATA_FOLDER}/subject-info.csv"

RESAMPLE_TO = 128
RANDOM_STATE = 42

WINDOW_SEC = 2.0
OVERLAP_RATIO = 0.75

USE_BASELINE = True

BATCH_SIZE = 128
EPOCHS = 40
LR = 1e-3
PATIENCE = 6

# BiLSTM
HIDDEN = 64
NUM_LAYERS = 2
DROPOUT = 0.3

# ======================
# Seed
# ======================
def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
seed_everything(RANDOM_STATE)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)


# ======================
# لود داده + baseline correction
# ======================
def load_task_edf_with_baseline(folder_path, info_csv_path, resample_to=None,
                                use_baseline=True):
    folder = Path(folder_path)
    if not folder.is_dir():
        raise NotADirectoryError(f"{folder_path} is not a valid directory")

    info_df = pd.read_csv(info_csv_path)
    label_map = dict(zip(info_df["Subject"], info_df["Count quality"]))

    X_list, y_list, subjects = [], [], []
    sfreq = None

    for subj_name in info_df["Subject"]:
        task_file = folder / f"{subj_name}_2.edf"
        rest_file = folder / f"{subj_name}_1.edf"

        if not task_file.is_file():
            print(f"Task file not found for {subj_name}, skipping.")
            continue

        if subj_name not in label_map:
            print(f"{subj_name} not in subject-info, skipping.")
            continue

        raw_task = mne.io.read_raw_edf(task_file, preload=True, verbose=False)

        raw_rest = None
        if use_baseline and rest_file.is_file():
            raw_rest = mne.io.read_raw_edf(rest_file, preload=True, verbose=False)

        if resample_to is not None:
            raw_task.resample(resample_to)
            if raw_rest is not None:
                raw_rest.resample(resample_to)

        if sfreq is None:
            sfreq = raw_task.info["sfreq"]

        task_data = raw_task.get_data()  # (C, T_task)

        if use_baseline and raw_rest is not None:
            rest_data = raw_rest.get_data()  # (C, T_rest)
            baseline = rest_data.mean(axis=1, keepdims=True)
            task_data = task_data - baseline

        X_list.append(task_data)
        y_list.append(int(label_map[subj_name]))
        subjects.append(subj_name)

    if not X_list:
        raise ValueError("No files loaded. Check paths/names.")

    lengths = [d.shape[1] for d in X_list]
    min_len = min(lengths)

    X = np.stack([d[:, :min_len] for d in X_list], axis=0)  # (N, C, T)
    y = np.array(y_list, dtype=int)
    subjects = np.array(subjects)

    print("X:", X.shape, "y:", y.shape, "sfreq:", sfreq)
    return X, y, subjects, sfreq


# ======================
# Segment با overlap 75%
# ======================
def make_segments(X, y, subjects, sfreq, window_sec=2.0, overlap_ratio=0.75):
    N, C, T = X.shape
    win_size = int(window_sec * sfreq)          # 2s * 128 = 256
    step_sec = window_sec * (1.0 - overlap_ratio)
    step = max(1, int(step_sec * sfreq))        # 0.5s * 128 = 64

    seg_X, seg_y, seg_subj = [], [], []

    for i in range(N):
        data = X[i]  # (C, T)
        lab = y[i]
        sid = subjects[i]

        start = 0
        while start + win_size <= T:
            seg = data[:, start:start+win_size]  # (C, win)
            seg_X.append(seg)
            seg_y.append(lab)
            seg_subj.append(sid)
            start += step

    seg_X = np.stack(seg_X, axis=0)  # (Nseg, C, win)
    seg_y = np.array(seg_y, dtype=int)
    seg_subj = np.array(seg_subj)

    print("Segments:", seg_X.shape)
    return seg_X, seg_y, seg_subj


# ======================
# Dataset + Augmentation (فقط train)
# ======================
class EEGSegDataset(Dataset):
    def __init__(self, X_seq, y, augment=False):
        """
        X_seq: (N, T, C) float32
        y: (N,) int
        """
        self.X = X_seq.astype(np.float32)
        self.y = y.astype(np.float32)
        self.augment = augment

    def __len__(self):
        return len(self.y)

    def _augment(self, x):
        # x: (T, C)
        # 1) noise
        if np.random.rand() < 0.5:
            noise = np.random.normal(0, 0.02, size=x.shape).astype(np.float32)
            x = x + noise
        # 2) channel-wise scale
        if np.random.rand() < 0.5:
            scale = (1.0 + np.random.normal(0, 0.05, size=(1, x.shape[1]))).astype(np.float32)
            x = x * scale
        return x

    def __getitem__(self, idx):
        x = self.X[idx]
        if self.augment:
            x = self._augment(x.copy())
        y = self.y[idx]
        return torch.from_numpy(x), torch.tensor(y)


# ======================
# BiLSTM Model
# ======================
class BiLSTMClassifier(nn.Module):
    def __init__(self, n_channels, hidden=64, num_layers=2, dropout=0.3):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=n_channels,
            hidden_size=hidden,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True,
            dropout=dropout if num_layers > 1 else 0.0
        )
        self.norm = nn.LayerNorm(hidden * 2)
        self.fc = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(hidden * 2, 64),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        # x: (B, T, C)
        out, _ = self.lstm(x)       # (B, T, 2H)
        out = out.mean(dim=1)       # mean pooling over time -> (B, 2H)
        out = self.norm(out)
        logits = self.fc(out).squeeze(1)  # (B,)
        return logits


# ======================
# Utilities: normalization without leakage
# ======================
def compute_train_norm_stats(X_train_seq):
    # X_train_seq: (N, T, C)
    flat = X_train_seq.reshape(-1, X_train_seq.shape[-1])  # (N*T, C)
    mean = flat.mean(axis=0, keepdims=True)
    std = flat.std(axis=0, keepdims=True) + 1e-6
    return mean.astype(np.float32), std.astype(np.float32)

def apply_norm(X_seq, mean, std):
    return ((X_seq - mean) / std).astype(np.float32)


# ======================
# Train/Val split by SUBJECT (برای LOSO بهتر)
# ======================
def make_subject_val_split(train_subjects, val_ratio=0.15):
    uniq = np.unique(train_subjects)
    rng = np.random.RandomState(RANDOM_STATE)
    rng.shuffle(uniq)
    n_val = max(1, int(len(uniq) * val_ratio))
    val_subj = set(uniq[:n_val])
    val_mask = np.array([s in val_subj for s in train_subjects])
    return val_mask


# ======================
# Training loop with early stopping
# ======================
def train_one_fold(X_train, y_train, subj_train, n_channels):
    # val split by subject
    val_mask = make_subject_val_split(subj_train, val_ratio=0.15)
    tr_mask = ~val_mask

    X_tr, y_tr = X_train[tr_mask], y_train[tr_mask]
    X_val, y_val = X_train[val_mask], y_train[val_mask]

    # class imbalance handling
    n_pos = (y_tr == 1).sum()
    n_neg = (y_tr == 0).sum()
    pos_weight = torch.tensor([n_neg / max(1, n_pos)], device=device, dtype=torch.float32)

    # sampler (optional) - کمک می‌کند batchها متعادل‌تر شوند
    weights = np.where(y_tr == 1, n_neg / max(1, n_pos), 1.0).astype(np.float32)
    sampler = WeightedRandomSampler(weights=weights, num_samples=len(weights), replacement=True)

    ds_tr = EEGSegDataset(X_tr, y_tr, augment=True)
    ds_val = EEGSegDataset(X_val, y_val, augment=False)

    dl_tr = DataLoader(ds_tr, batch_size=BATCH_SIZE, sampler=sampler, num_workers=2, pin_memory=True)
    dl_val = DataLoader(ds_val, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

    model = BiLSTMClassifier(n_channels=n_channels, hidden=HIDDEN, num_layers=NUM_LAYERS, dropout=DROPOUT).to(device)
    opt = torch.optim.Adam(model.parameters(), lr=LR)
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

    best_f1 = -1
    best_state = None
    patience = 0

    for epoch in range(1, EPOCHS + 1):
        model.train()
        tr_losses = []

        for xb, yb in dl_tr:
            xb = xb.to(device)
            yb = yb.to(device)

            opt.zero_grad()
            logits = model(xb)
            loss = criterion(logits, yb)
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            opt.step()
            tr_losses.append(loss.item())

        # validation
        model.eval()
        all_pred, all_true = [], []
        with torch.no_grad():
            for xb, yb in dl_val:
                xb = xb.to(device)
                logits = model(xb)
                prob = torch.sigmoid(logits).cpu().numpy()
                pred = (prob >= 0.5).astype(int)
                all_pred.append(pred)
                all_true.append(yb.numpy().astype(int))

        all_pred = np.concatenate(all_pred)
        all_true = np.concatenate(all_true)

        f1 = f1_score(all_true, all_pred, zero_division=0)
        avg_loss = float(np.mean(tr_losses))

        # print short progress
        if epoch == 1 or epoch % 5 == 0:
            print(f"  Epoch {epoch:02d} | train_loss={avg_loss:.4f} | val_f1={f1:.4f}")

        if f1 > best_f1:
            best_f1 = f1
            best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
            patience = 0
        else:
            patience += 1
            if patience >= PATIENCE:
                break

    # load best
    model.load_state_dict(best_state)
    model.eval()
    return model


# ======================
# LOSO Evaluation
# ======================
def evaluate_loso_bilstm(seg_X, seg_y, seg_subjects):
    # تبدیل به (N, T, C)
    X_seq = np.transpose(seg_X, (0, 2, 1)).astype(np.float32)  # (Nseg, 256, 21)
    y = seg_y.astype(int)
    subjects = seg_subjects

    uniq_subj = np.unique(subjects)
    print("Unique subjects:", len(uniq_subj))

    seg_metrics = []
    subj_metrics = []

    for test_subj in uniq_subj:
        print("\n" + "="*60)
        print("Test subject:", test_subj)

        test_mask = (subjects == test_subj)
        train_mask = ~test_mask

        X_train_raw = X_seq[train_mask]
        y_train = y[train_mask]
        subj_train = subjects[train_mask]

        X_test_raw = X_seq[test_mask]
        y_test = y[test_mask]

        # normalization from TRAIN only (NO leakage)
        mean, std = compute_train_norm_stats(X_train_raw)
        X_train = apply_norm(X_train_raw, mean, std)
        X_test  = apply_norm(X_test_raw,  mean, std)

        print("Train seg:", X_train.shape[0], "| Test seg:", X_test.shape[0],
              "| Train dist:", np.bincount(y_train))

        # train
        model = train_one_fold(X_train, y_train, subj_train, n_channels=X_train.shape[-1])

        # predict test
        with torch.no_grad():
            xb = torch.from_numpy(X_test).to(device)
            logits = model(xb).cpu().numpy()
            prob = 1.0 / (1.0 + np.exp(-logits))
            y_pred = (prob >= 0.5).astype(int)

        # segment-level metrics (برای باینری)
        acc = accuracy_score(y_test, y_pred)
        prec = precision_score(y_test, y_pred, zero_division=0)
        rec = recall_score(y_test, y_pred, zero_division=0)
        f1 = f1_score(y_test, y_pred, zero_division=0)
        print(f"Segment-level -> acc={acc:.4f}, prec={prec:.4f}, rec={rec:.4f}, f1={f1:.4f}")
        seg_metrics.append([acc, prec, rec, f1])

        # subject-level majority vote
        true_label = int(y_test[0])
        pred_label = int(np.argmax(np.bincount(y_pred)))
        acc_subj = 1.0 if pred_label == true_label else 0.0
        print(f"Subject-level -> true={true_label}, pred={pred_label}, acc={acc_subj:.4f}")
        subj_metrics.append([acc_subj, acc_subj, acc_subj, acc_subj])

    seg_metrics = np.array(seg_metrics)
    subj_metrics = np.array(subj_metrics)

    print("\n" + "="*60)
    print("Average SEGMENT-level (LOSO):")
    print(f"Accuracy : {seg_metrics[:,0].mean():.4f}")
    print(f"Precision: {seg_metrics[:,1].mean():.4f}")
    print(f"Recall   : {seg_metrics[:,2].mean():.4f}")
    print(f"F1-score : {seg_metrics[:,3].mean():.4f}")

    print("\n" + "="*60)
    print("Average SUBJECT-level (LOSO, majority vote):")
    print(f"Accuracy : {subj_metrics[:,0].mean():.4f}")
    print(f"Precision: {subj_metrics[:,1].mean():.4f}")
    print(f"Recall   : {subj_metrics[:,2].mean():.4f}")
    print(f"F1-score : {subj_metrics[:,3].mean():.4f}")


# ======================
# main
# ======================
def main():
    X, y, subjects, sfreq = load_task_edf_with_baseline(
        DATA_FOLDER, INFO_CSV,
        resample_to=RESAMPLE_TO,
        use_baseline=USE_BASELINE
    )

    seg_X, seg_y, seg_subjects = make_segments(
        X, y, subjects, sfreq,
        window_sec=WINDOW_SEC,
        overlap_ratio=OVERLAP_RATIO
    )

    evaluate_loso_bilstm(seg_X, seg_y, seg_subjects)

if __name__ == "__main__":
    main()


Device: cuda
X: (36, 21, 7936) y: (36,) sfreq: 128.0
Segments: (4356, 21, 256)
Unique subjects: 36

Test subject: Subject00
Train seg: 4235 | Test seg: 121 | Train dist: [1089 3146]
  Epoch 01 | train_loss=0.3623 | val_f1=0.0000
  Epoch 05 | train_loss=0.0354 | val_f1=0.4084
Segment-level -> acc=0.0165, prec=0.0000, rec=0.0000, f1=0.0000
Subject-level -> true=0, pred=1, acc=0.0000

Test subject: Subject01
Train seg: 4235 | Test seg: 121 | Train dist: [1210 3025]
  Epoch 01 | train_loss=0.4089 | val_f1=0.0000
  Epoch 05 | train_loss=0.0714 | val_f1=0.2044
  Epoch 10 | train_loss=0.0214 | val_f1=0.2188
Segment-level -> acc=0.9008, prec=1.0000, rec=0.9008, f1=0.9478
Subject-level -> true=1, pred=1, acc=1.0000

Test subject: Subject02
Train seg: 4235 | Test seg: 121 | Train dist: [1210 3025]
  Epoch 01 | train_loss=0.4044 | val_f1=0.0000
  Epoch 05 | train_loss=0.1025 | val_f1=0.3007
Segment-level -> acc=0.9835, prec=1.0000, rec=0.9835, f1=0.9917
Subject-level -> true=1, pred=1, acc=1.0000

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import random
import warnings
warnings.filterwarnings("ignore")

import mne
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# ======================
# تنظیمات کلی
# ======================
DATA_FOLDER = "/kaggle/input/ahmadi-dataset"
INFO_CSV = f"{DATA_FOLDER}/subject-info.csv"

RESAMPLE_TO = 128
RANDOM_STATE = 42

WINDOW_SEC = 2.0
OVERLAP_RATIO = 0.75

USE_BASELINE = True

BATCH_SIZE = 128
EPOCHS = 40
LR = 1e-3
PATIENCE = 6

# BiLSTM
HIDDEN = 64
NUM_LAYERS = 2
DROPOUT = 0.3

# --- Validation by SUBJECT
VAL_SUBJECT_COUNT = 2          # دقیقاً 2 سابجکت برای ولید (از trainها)
NOISY_VALIDATION = True       # حالت 2: ولید نویزی
VAL_NOISE_STD = 0.05           # شدت نویز برای ولید (اگر NOISY_VALIDATION=True)
VAL_SCALE_STD = 0.08           # شدت scale-jitter برای ولید نویزی (اختیاری)

# ======================
# Seed
# ======================
def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

seed_everything(RANDOM_STATE)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)


# ======================
# لود داده + baseline correction
# ======================
def load_task_edf_with_baseline(folder_path, info_csv_path, resample_to=None,
                                use_baseline=True):
    folder = Path(folder_path)
    if not folder.is_dir():
        raise NotADirectoryError(f"{folder_path} is not a valid directory")

    info_df = pd.read_csv(info_csv_path)
    label_map = dict(zip(info_df["Subject"], info_df["Count quality"]))

    X_list, y_list, subjects = [], [], []
    sfreq = None

    for subj_name in info_df["Subject"]:
        task_file = folder / f"{subj_name}_2.edf"
        rest_file = folder / f"{subj_name}_1.edf"

        if not task_file.is_file():
            print(f"Task file not found for {subj_name}, skipping.")
            continue

        raw_task = mne.io.read_raw_edf(task_file, preload=True, verbose=False)

        raw_rest = None
        if use_baseline and rest_file.is_file():
            raw_rest = mne.io.read_raw_edf(rest_file, preload=True, verbose=False)

        if resample_to is not None:
            raw_task.resample(resample_to)
            if raw_rest is not None:
                raw_rest.resample(resample_to)

        if sfreq is None:
            sfreq = raw_task.info["sfreq"]

        task_data = raw_task.get_data()  # (C, T_task)

        if use_baseline and raw_rest is not None:
            rest_data = raw_rest.get_data()  # (C, T_rest)
            baseline = rest_data.mean(axis=1, keepdims=True)
            task_data = task_data - baseline

        X_list.append(task_data)
        y_list.append(int(label_map[subj_name]))
        subjects.append(subj_name)

    if not X_list:
        raise ValueError("No files loaded. Check paths/names.")

    lengths = [d.shape[1] for d in X_list]
    min_len = min(lengths)

    X = np.stack([d[:, :min_len] for d in X_list], axis=0)  # (N, C, T)
    y = np.array(y_list, dtype=int)
    subjects = np.array(subjects)

    print("X:", X.shape, "y:", y.shape, "sfreq:", sfreq)
    return X, y, subjects, sfreq


# ======================
# Segment با overlap 75%
# ======================
def make_segments(X, y, subjects, sfreq, window_sec=2.0, overlap_ratio=0.75):
    N, C, T = X.shape
    win_size = int(window_sec * sfreq)          # 256
    step_sec = window_sec * (1.0 - overlap_ratio)
    step = max(1, int(step_sec * sfreq))        # 64

    seg_X, seg_y, seg_subj = [], [], []

    for i in range(N):
        data = X[i]  # (C, T)
        lab = y[i]
        sid = subjects[i]

        start = 0
        while start + win_size <= T:
            seg = data[:, start:start+win_size]  # (C, win)
            seg_X.append(seg)
            seg_y.append(lab)
            seg_subj.append(sid)
            start += step

    seg_X = np.stack(seg_X, axis=0)  # (Nseg, C, win)
    seg_y = np.array(seg_y, dtype=int)
    seg_subj = np.array(seg_subj)

    print("Segments:", seg_X.shape)
    return seg_X, seg_y, seg_subj


# ======================
# Dataset + Augmentation
# ======================
class EEGSegDataset(Dataset):
    def __init__(self, X_seq, y, do_noise=False, do_scale=False,
                 noise_std=0.02, scale_std=0.05):
        """
        X_seq: (N, T, C) float32
        y: (N,) int/float
        """
        self.X = X_seq.astype(np.float32)
        self.y = y.astype(np.float32)

        self.do_noise = do_noise
        self.do_scale = do_scale
        self.noise_std = float(noise_std)
        self.scale_std = float(scale_std)

    def __len__(self):
        return len(self.y)

    def _augment(self, x):
        # x: (T, C)
        if self.do_noise:
            noise = np.random.normal(0, self.noise_std, size=x.shape).astype(np.float32)
            x = x + noise
        if self.do_scale:
            scale = (1.0 + np.random.normal(0, self.scale_std, size=(1, x.shape[1]))).astype(np.float32)
            x = x * scale
        return x

    def __getitem__(self, idx):
        x = self.X[idx]
        if self.do_noise or self.do_scale:
            x = self._augment(x.copy())
        y = self.y[idx]
        return torch.from_numpy(x), torch.tensor(y)


# ======================
# BiLSTM Model
# ======================
class BiLSTMClassifier(nn.Module):
    def __init__(self, n_channels, hidden=64, num_layers=2, dropout=0.3):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=n_channels,
            hidden_size=hidden,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True,
            dropout=dropout if num_layers > 1 else 0.0
        )
        self.norm = nn.LayerNorm(hidden * 2)
        self.fc = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(hidden * 2, 64),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        # x: (B, T, C)
        out, _ = self.lstm(x)       # (B, T, 2H)
        out = out.mean(dim=1)       # mean pooling over time -> (B, 2H)
        out = self.norm(out)
        logits = self.fc(out).squeeze(1)  # (B,)
        return logits


# ======================
# Normalization without leakage
# ======================
def compute_train_norm_stats(X_train_seq):
    flat = X_train_seq.reshape(-1, X_train_seq.shape[-1])  # (N*T, C)
    mean = flat.mean(axis=0, keepdims=True)
    std = flat.std(axis=0, keepdims=True) + 1e-6
    return mean.astype(np.float32), std.astype(np.float32)

def apply_norm(X_seq, mean, std):
    return ((X_seq - mean) / std).astype(np.float32)


# ======================
# Val split: دقیقاً K سابجکت رندوم از train
# ======================
def make_subject_val_split_exact_k(train_subjects, k=2, seed=42):
    uniq = np.unique(train_subjects)
    if len(uniq) <= k:
        # اگر تعداد سابجکت‌های train کم بود، مجبوریم 1 تا ولید کنیم
        k = max(1, len(uniq) - 1)

    rng = np.random.RandomState(seed)
    rng.shuffle(uniq)
    val_subj = set(uniq[:k])
    val_mask = np.array([s in val_subj for s in train_subjects])
    return val_mask, val_subj


# ======================
# Train with early stopping
# ======================
def train_one_fold(X_train, y_train, subj_train, n_channels, fold_seed,
                   noisy_validation=False):
    # validation: 2 subject random
    val_mask, val_subj = make_subject_val_split_exact_k(
        subj_train, k=VAL_SUBJECT_COUNT, seed=fold_seed
    )
    tr_mask = ~val_mask

    X_tr, y_tr = X_train[tr_mask], y_train[tr_mask]
    X_val, y_val = X_train[val_mask], y_train[val_mask]

    # class imbalance
    n_pos = int((y_tr == 1).sum())
    n_neg = int((y_tr == 0).sum())
    pos_weight = torch.tensor([n_neg / max(1, n_pos)], device=device, dtype=torch.float32)

    # sampler (balanced batches)
    weights = np.where(y_tr == 1, n_neg / max(1, n_pos), 1.0).astype(np.float32)
    sampler = WeightedRandomSampler(weights=weights, num_samples=len(weights), replacement=True)

    # Train augmentation (معمولاً مفید برای LOSO)
    ds_tr = EEGSegDataset(
        X_tr, y_tr,
        do_noise=True, do_scale=True,
        noise_std=0.02, scale_std=0.05
    )

    # Validation:
    # - حالت معمول: بدون augmentation
    # - حالت نویزی: عمداً ولید را نویزی می‌کنیم
    if noisy_validation:
        ds_val = EEGSegDataset(
            X_val, y_val,
            do_noise=True, do_scale=True,
            noise_std=VAL_NOISE_STD, scale_std=VAL_SCALE_STD
        )
    else:
        ds_val = EEGSegDataset(X_val, y_val, do_noise=False, do_scale=False)

    dl_tr = DataLoader(ds_tr, batch_size=BATCH_SIZE, sampler=sampler, num_workers=2, pin_memory=True)
    dl_val = DataLoader(ds_val, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

    model = BiLSTMClassifier(
        n_channels=n_channels, hidden=HIDDEN, num_layers=NUM_LAYERS, dropout=DROPOUT
    ).to(device)

    opt = torch.optim.Adam(model.parameters(), lr=LR)
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

    best_f1 = -1
    best_state = None
    patience = 0

    print(f"  Validation subjects: {sorted(list(val_subj))} | noisy_val={noisy_validation}")

    for epoch in range(1, EPOCHS + 1):
        model.train()
        tr_losses = []

        for xb, yb in dl_tr:
            xb = xb.to(device)
            yb = yb.to(device)

            opt.zero_grad()
            logits = model(xb)
            loss = criterion(logits, yb)
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            opt.step()
            tr_losses.append(loss.item())

        # validation
        model.eval()
        all_pred, all_true = [], []
        with torch.no_grad():
            for xb, yb in dl_val:
                xb = xb.to(device)
                logits = model(xb)
                prob = torch.sigmoid(logits).cpu().numpy()
                pred = (prob >= 0.5).astype(int)
                all_pred.append(pred)
                all_true.append(yb.numpy().astype(int))

        all_pred = np.concatenate(all_pred)
        all_true = np.concatenate(all_true)

        f1 = f1_score(all_true, all_pred, zero_division=0)
        avg_loss = float(np.mean(tr_losses))

        if epoch == 1 or epoch % 5 == 0:
            print(f"  Epoch {epoch:02d} | train_loss={avg_loss:.4f} | val_f1={f1:.4f}")

        if f1 > best_f1:
            best_f1 = f1
            best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
            patience = 0
        else:
            patience += 1
            if patience >= PATIENCE:
                break

    model.load_state_dict(best_state)
    model.eval()
    return model


# ======================
# LOSO Evaluation
# ======================
def evaluate_loso_bilstm(seg_X, seg_y, seg_subjects, noisy_validation=False):
    # (N, C, T) -> (N, T, C)
    X_seq = np.transpose(seg_X, (0, 2, 1)).astype(np.float32)  # (Nseg, 256, 21)
    y = seg_y.astype(int)
    subjects = seg_subjects

    uniq_subj = np.unique(subjects)
    print("Unique subjects:", len(uniq_subj))

    seg_metrics = []
    subj_metrics = []

    for test_subj in uniq_subj:
        print("\n" + "="*60)
        print("Test subject:", test_subj)

        test_mask = (subjects == test_subj)
        train_mask = ~test_mask

        X_train_raw = X_seq[train_mask]
        y_train = y[train_mask]
        subj_train = subjects[train_mask]

        X_test_raw = X_seq[test_mask]
        y_test = y[test_mask]

        # normalization from TRAIN only
        mean, std = compute_train_norm_stats(X_train_raw)
        X_train = apply_norm(X_train_raw, mean, std)
        X_test  = apply_norm(X_test_raw,  mean, std)

        print("Train seg:", X_train.shape[0], "| Test seg:", X_test.shape[0],
              "| Train dist:", np.bincount(y_train))

        # fold-specific seed (برای اینکه انتخاب 2 سابجکت ولید هر fold رندوم ولی reproducible باشد)
        # SubjectXX -> XX
        try:
            sid_num = int(str(test_subj).replace("Subject", ""))
        except:
            sid_num = 0
        fold_seed = RANDOM_STATE + 1000 + sid_num

        model = train_one_fold(
            X_train, y_train, subj_train,
            n_channels=X_train.shape[-1],
            fold_seed=fold_seed,
            noisy_validation=noisy_validation
        )

        # predict test
        with torch.no_grad():
            xb = torch.from_numpy(X_test).to(device)
            logits = model(xb).cpu().numpy()
            prob = 1.0 / (1.0 + np.exp(-logits))
            y_pred = (prob >= 0.5).astype(int)

        # segment-level metrics
        acc = accuracy_score(y_test, y_pred)
        prec = precision_score(y_test, y_pred, zero_division=0)
        rec = recall_score(y_test, y_pred, zero_division=0)
        f1 = f1_score(y_test, y_pred, zero_division=0)
        print(f"Segment-level -> acc={acc:.4f}, prec={prec:.4f}, rec={rec:.4f}, f1={f1:.4f}")
        seg_metrics.append([acc, prec, rec, f1])

        # subject-level majority vote
        true_label = int(y_test[0])
        pred_label = int(np.argmax(np.bincount(y_pred)))
        acc_subj = 1.0 if pred_label == true_label else 0.0
        print(f"Subject-level -> true={true_label}, pred={pred_label}, acc={acc_subj:.4f}")
        subj_metrics.append([acc_subj, acc_subj, acc_subj, acc_subj])

    seg_metrics = np.array(seg_metrics)
    subj_metrics = np.array(subj_metrics)

    print("\n" + "="*60)
    print("Average SEGMENT-level (LOSO):")
    print(f"Accuracy : {seg_metrics[:,0].mean():.4f}")
    print(f"Precision: {seg_metrics[:,1].mean():.4f}")
    print(f"Recall   : {seg_metrics[:,2].mean():.4f}")
    print(f"F1-score : {seg_metrics[:,3].mean():.4f}")

    print("\n" + "="*60)
    print("Average SUBJECT-level (LOSO, majority vote):")
    print(f"Accuracy : {subj_metrics[:,0].mean():.4f}")
    print(f"Precision: {subj_metrics[:,1].mean():.4f}")
    print(f"Recall   : {subj_metrics[:,2].mean():.4f}")
    print(f"F1-score : {subj_metrics[:,3].mean():.4f}")


# ======================
# main
# ======================
def main():
    X, y, subjects, sfreq = load_task_edf_with_baseline(
        DATA_FOLDER, INFO_CSV,
        resample_to=RESAMPLE_TO,
        use_baseline=USE_BASELINE
    )

    seg_X, seg_y, seg_subjects = make_segments(
        X, y, subjects, sfreq,
        window_sec=WINDOW_SEC,
        overlap_ratio=OVERLAP_RATIO
    )

    evaluate_loso_bilstm(seg_X, seg_y, seg_subjects, noisy_validation=NOISY_VALIDATION)

if __name__ == "__main__":
    main()


In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import random
import warnings
warnings.filterwarnings("ignore")

import mne
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# ======================
# تنظیمات کلی
# ======================
DATA_FOLDER = "/kaggle/input/ahmadi-dataset"
INFO_CSV = f"{DATA_FOLDER}/subject-info.csv"

RESAMPLE_TO = 128
RANDOM_STATE = 42

WINDOW_SEC = 2.0
OVERLAP_RATIO = 0.75

USE_BASELINE = True

BATCH_SIZE = 128
EPOCHS = 40
LR = 1e-3
PATIENCE = 6

# BiLSTM
HIDDEN = 64
NUM_LAYERS = 2
DROPOUT = 0.3

# --- Validation by SUBJECT
VAL_SUBJECT_COUNT = 4          # دقیقاً 2 سابجکت برای ولید (از trainها)
NOISY_VALIDATION = False       # حالت 2: ولید نویزی
VAL_NOISE_STD = 0.05           # شدت نویز برای ولید (اگر NOISY_VALIDATION=True)
VAL_SCALE_STD = 0.08           # شدت scale-jitter برای ولید نویزی (اختیاری)

# ======================
# Seed
# ======================
def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

seed_everything(RANDOM_STATE)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)


# ======================
# لود داده + baseline correction
# ======================
def load_task_edf_with_baseline(folder_path, info_csv_path, resample_to=None,
                                use_baseline=True):
    folder = Path(folder_path)
    if not folder.is_dir():
        raise NotADirectoryError(f"{folder_path} is not a valid directory")

    info_df = pd.read_csv(info_csv_path)
    label_map = dict(zip(info_df["Subject"], info_df["Count quality"]))

    X_list, y_list, subjects = [], [], []
    sfreq = None

    for subj_name in info_df["Subject"]:
        task_file = folder / f"{subj_name}_2.edf"
        rest_file = folder / f"{subj_name}_1.edf"

        if not task_file.is_file():
            print(f"Task file not found for {subj_name}, skipping.")
            continue

        raw_task = mne.io.read_raw_edf(task_file, preload=True, verbose=False)

        raw_rest = None
        if use_baseline and rest_file.is_file():
            raw_rest = mne.io.read_raw_edf(rest_file, preload=True, verbose=False)

        if resample_to is not None:
            raw_task.resample(resample_to)
            if raw_rest is not None:
                raw_rest.resample(resample_to)

        if sfreq is None:
            sfreq = raw_task.info["sfreq"]

        task_data = raw_task.get_data()  # (C, T_task)

        if use_baseline and raw_rest is not None:
            rest_data = raw_rest.get_data()  # (C, T_rest)
            baseline = rest_data.mean(axis=1, keepdims=True)
            task_data = task_data - baseline

        X_list.append(task_data)
        y_list.append(int(label_map[subj_name]))
        subjects.append(subj_name)

    if not X_list:
        raise ValueError("No files loaded. Check paths/names.")

    lengths = [d.shape[1] for d in X_list]
    min_len = min(lengths)

    X = np.stack([d[:, :min_len] for d in X_list], axis=0)  # (N, C, T)
    y = np.array(y_list, dtype=int)
    subjects = np.array(subjects)

    print("X:", X.shape, "y:", y.shape, "sfreq:", sfreq)
    return X, y, subjects, sfreq


# ======================
# Segment با overlap 75%
# ======================
def make_segments(X, y, subjects, sfreq, window_sec=2.0, overlap_ratio=0.75):
    N, C, T = X.shape
    win_size = int(window_sec * sfreq)          # 256
    step_sec = window_sec * (1.0 - overlap_ratio)
    step = max(1, int(step_sec * sfreq))        # 64

    seg_X, seg_y, seg_subj = [], [], []

    for i in range(N):
        data = X[i]  # (C, T)
        lab = y[i]
        sid = subjects[i]

        start = 0
        while start + win_size <= T:
            seg = data[:, start:start+win_size]  # (C, win)
            seg_X.append(seg)
            seg_y.append(lab)
            seg_subj.append(sid)
            start += step

    seg_X = np.stack(seg_X, axis=0)  # (Nseg, C, win)
    seg_y = np.array(seg_y, dtype=int)
    seg_subj = np.array(seg_subj)

    print("Segments:", seg_X.shape)
    return seg_X, seg_y, seg_subj


# ======================
# Dataset + Augmentation
# ======================
class EEGSegDataset(Dataset):
    def __init__(self, X_seq, y, do_noise=False, do_scale=False,
                 noise_std=0.02, scale_std=0.05):
        """
        X_seq: (N, T, C) float32
        y: (N,) int/float
        """
        self.X = X_seq.astype(np.float32)
        self.y = y.astype(np.float32)

        self.do_noise = do_noise
        self.do_scale = do_scale
        self.noise_std = float(noise_std)
        self.scale_std = float(scale_std)

    def __len__(self):
        return len(self.y)

    def _augment(self, x):
        # x: (T, C)
        if self.do_noise:
            noise = np.random.normal(0, self.noise_std, size=x.shape).astype(np.float32)
            x = x + noise
        if self.do_scale:
            scale = (1.0 + np.random.normal(0, self.scale_std, size=(1, x.shape[1]))).astype(np.float32)
            x = x * scale
        return x

    def __getitem__(self, idx):
        x = self.X[idx]
        if self.do_noise or self.do_scale:
            x = self._augment(x.copy())
        y = self.y[idx]
        return torch.from_numpy(x), torch.tensor(y)


# ======================
# BiLSTM Model
# ======================
class BiLSTMClassifier(nn.Module):
    def __init__(self, n_channels, hidden=64, num_layers=2, dropout=0.3):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=n_channels,
            hidden_size=hidden,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True,
            dropout=dropout if num_layers > 1 else 0.0
        )
        self.norm = nn.LayerNorm(hidden * 2)
        self.fc = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(hidden * 2, 64),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        # x: (B, T, C)
        out, _ = self.lstm(x)       # (B, T, 2H)
        out = out.mean(dim=1)       # mean pooling over time -> (B, 2H)
        out = self.norm(out)
        logits = self.fc(out).squeeze(1)  # (B,)
        return logits


# ======================
# Normalization without leakage
# ======================
def compute_train_norm_stats(X_train_seq):
    flat = X_train_seq.reshape(-1, X_train_seq.shape[-1])  # (N*T, C)
    mean = flat.mean(axis=0, keepdims=True)
    std = flat.std(axis=0, keepdims=True) + 1e-6
    return mean.astype(np.float32), std.astype(np.float32)

def apply_norm(X_seq, mean, std):
    return ((X_seq - mean) / std).astype(np.float32)


# ======================
# Val split: دقیقاً K سابجکت رندوم از train
# ======================
def make_subject_val_split_exact_k(train_subjects, k=2, seed=42):
    uniq = np.unique(train_subjects)
    if len(uniq) <= k:
        # اگر تعداد سابجکت‌های train کم بود، مجبوریم 1 تا ولید کنیم
        k = max(1, len(uniq) - 1)

    rng = np.random.RandomState(seed)
    rng.shuffle(uniq)
    val_subj = set(uniq[:k])
    val_mask = np.array([s in val_subj for s in train_subjects])
    return val_mask, val_subj


# ======================
# Train with early stopping
# ======================
def train_one_fold(X_train, y_train, subj_train, n_channels, fold_seed,
                   noisy_validation=False):
    # validation: 2 subject random
    val_mask, val_subj = make_subject_val_split_exact_k(
        subj_train, k=VAL_SUBJECT_COUNT, seed=fold_seed
    )
    tr_mask = ~val_mask

    X_tr, y_tr = X_train[tr_mask], y_train[tr_mask]
    X_val, y_val = X_train[val_mask], y_train[val_mask]

    # class imbalance
    n_pos = int((y_tr == 1).sum())
    n_neg = int((y_tr == 0).sum())
    pos_weight = torch.tensor([n_neg / max(1, n_pos)], device=device, dtype=torch.float32)

    # sampler (balanced batches)
    weights = np.where(y_tr == 1, n_neg / max(1, n_pos), 1.0).astype(np.float32)
    sampler = WeightedRandomSampler(weights=weights, num_samples=len(weights), replacement=True)

    # Train augmentation (معمولاً مفید برای LOSO)
    ds_tr = EEGSegDataset(
        X_tr, y_tr,
        do_noise=True, do_scale=True,
        noise_std=0.02, scale_std=0.05
    )

    # Validation:
    # - حالت معمول: بدون augmentation
    # - حالت نویزی: عمداً ولید را نویزی می‌کنیم
    if noisy_validation:
        ds_val = EEGSegDataset(
            X_val, y_val,
            do_noise=True, do_scale=True,
            noise_std=VAL_NOISE_STD, scale_std=VAL_SCALE_STD
        )
    else:
        ds_val = EEGSegDataset(X_val, y_val, do_noise=False, do_scale=False)

    dl_tr = DataLoader(ds_tr, batch_size=BATCH_SIZE, sampler=sampler, num_workers=2, pin_memory=True)
    dl_val = DataLoader(ds_val, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

    model = BiLSTMClassifier(
        n_channels=n_channels, hidden=HIDDEN, num_layers=NUM_LAYERS, dropout=DROPOUT
    ).to(device)

    opt = torch.optim.Adam(model.parameters(), lr=LR)
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

    best_f1 = -1
    best_state = None
    patience = 0

    print(f"  Validation subjects: {sorted(list(val_subj))} | noisy_val={noisy_validation}")

    for epoch in range(1, EPOCHS + 1):
        model.train()
        tr_losses = []

        for xb, yb in dl_tr:
            xb = xb.to(device)
            yb = yb.to(device)

            opt.zero_grad()
            logits = model(xb)
            loss = criterion(logits, yb)
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            opt.step()
            tr_losses.append(loss.item())

        # validation
        model.eval()
        all_pred, all_true = [], []
        with torch.no_grad():
            for xb, yb in dl_val:
                xb = xb.to(device)
                logits = model(xb)
                prob = torch.sigmoid(logits).cpu().numpy()
                pred = (prob >= 0.5).astype(int)
                all_pred.append(pred)
                all_true.append(yb.numpy().astype(int))

        all_pred = np.concatenate(all_pred)
        all_true = np.concatenate(all_true)

        f1 = f1_score(all_true, all_pred, zero_division=0)
        avg_loss = float(np.mean(tr_losses))

        if epoch == 1 or epoch % 5 == 0:
            print(f"  Epoch {epoch:02d} | train_loss={avg_loss:.4f} | val_f1={f1:.4f}")

        if f1 > best_f1:
            best_f1 = f1
            best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
            patience = 0
        else:
            patience += 1
            if patience >= PATIENCE:
                break

    model.load_state_dict(best_state)
    model.eval()
    return model


# ======================
# LOSO Evaluation
# ======================
def evaluate_loso_bilstm(seg_X, seg_y, seg_subjects, noisy_validation=False):
    # (N, C, T) -> (N, T, C)
    X_seq = np.transpose(seg_X, (0, 2, 1)).astype(np.float32)  # (Nseg, 256, 21)
    y = seg_y.astype(int)
    subjects = seg_subjects

    uniq_subj = np.unique(subjects)
    print("Unique subjects:", len(uniq_subj))

    seg_metrics = []
    subj_metrics = []

    for test_subj in uniq_subj:
        print("\n" + "="*60)
        print("Test subject:", test_subj)

        test_mask = (subjects == test_subj)
        train_mask = ~test_mask

        X_train_raw = X_seq[train_mask]
        y_train = y[train_mask]
        subj_train = subjects[train_mask]

        X_test_raw = X_seq[test_mask]
        y_test = y[test_mask]

        # normalization from TRAIN only
        mean, std = compute_train_norm_stats(X_train_raw)
        X_train = apply_norm(X_train_raw, mean, std)
        X_test  = apply_norm(X_test_raw,  mean, std)

        print("Train seg:", X_train.shape[0], "| Test seg:", X_test.shape[0],
              "| Train dist:", np.bincount(y_train))

        # fold-specific seed (برای اینکه انتخاب 2 سابجکت ولید هر fold رندوم ولی reproducible باشد)
        # SubjectXX -> XX
        try:
            sid_num = int(str(test_subj).replace("Subject", ""))
        except:
            sid_num = 0
        fold_seed = RANDOM_STATE + 1000 + sid_num

        model = train_one_fold(
            X_train, y_train, subj_train,
            n_channels=X_train.shape[-1],
            fold_seed=fold_seed,
            noisy_validation=noisy_validation
        )

        # predict test
        with torch.no_grad():
            xb = torch.from_numpy(X_test).to(device)
            logits = model(xb).cpu().numpy()
            prob = 1.0 / (1.0 + np.exp(-logits))
            y_pred = (prob >= 0.5).astype(int)

        # segment-level metrics
        acc = accuracy_score(y_test, y_pred)
        prec = precision_score(y_test, y_pred, zero_division=0)
        rec = recall_score(y_test, y_pred, zero_division=0)
        f1 = f1_score(y_test, y_pred, zero_division=0)
        print(f"Segment-level -> acc={acc:.4f}, prec={prec:.4f}, rec={rec:.4f}, f1={f1:.4f}")
        seg_metrics.append([acc, prec, rec, f1])

        # subject-level majority vote
        true_label = int(y_test[0])
        pred_label = int(np.argmax(np.bincount(y_pred)))
        acc_subj = 1.0 if pred_label == true_label else 0.0
        print(f"Subject-level -> true={true_label}, pred={pred_label}, acc={acc_subj:.4f}")
        subj_metrics.append([acc_subj, acc_subj, acc_subj, acc_subj])

    seg_metrics = np.array(seg_metrics)
    subj_metrics = np.array(subj_metrics)

    print("\n" + "="*60)
    print("Average SEGMENT-level (LOSO):")
    print(f"Accuracy : {seg_metrics[:,0].mean():.4f}")
    print(f"Precision: {seg_metrics[:,1].mean():.4f}")
    print(f"Recall   : {seg_metrics[:,2].mean():.4f}")
    print(f"F1-score : {seg_metrics[:,3].mean():.4f}")

    print("\n" + "="*60)
    print("Average SUBJECT-level (LOSO, majority vote):")
    print(f"Accuracy : {subj_metrics[:,0].mean():.4f}")
    print(f"Precision: {subj_metrics[:,1].mean():.4f}")
    print(f"Recall   : {subj_metrics[:,2].mean():.4f}")
    print(f"F1-score : {subj_metrics[:,3].mean():.4f}")


# ======================
# main
# ======================
def main():
    X, y, subjects, sfreq = load_task_edf_with_baseline(
        DATA_FOLDER, INFO_CSV,
        resample_to=RESAMPLE_TO,
        use_baseline=USE_BASELINE
    )

    seg_X, seg_y, seg_subjects = make_segments(
        X, y, subjects, sfreq,
        window_sec=WINDOW_SEC,
        overlap_ratio=OVERLAP_RATIO
    )

    evaluate_loso_bilstm(seg_X, seg_y, seg_subjects, noisy_validation=NOISY_VALIDATION)

if __name__ == "__main__":
    main()
